<h1 align=center><font size = 5>Segmenting, Clustering and Comparing Neighborhoods from  New York City and London</font></h1>

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [2]:
import os
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from time import sleep

import requests # library to handle requests
import json # librart to handle json files

# Matplotlib and associated plotting modules

from matplotlib import pyplot as plt
plt.style.use('ggplot')

import matplotlib.cm as cm
import matplotlib.colors as colors


# import k-means from clustering stage
from sklearn.cluster import KMeans

# import to encode categorized features
from sklearn import preprocessing

import folium # map rendering library

from unidecode import unidecode

from lxml import html

print('Libraries imported.')

Libraries imported.


<a id='item1'></a>

## 1. Download and Explore Dataset

#### Load and explore the data

In [3]:
try:
    nyc_data = requests.get("https://cocl.us/new_york_dataset").json()
except Exception:
    with open('newyork_data.json', 'r') as json_data:
        nyc_data = json.load(json_data)
print('Data downloaded!')

Data downloaded!


All the relevant data is in the *features* key, which is basically a list of the neighborhoods. So, let's define a new variable that includes this data.

In [4]:
neighborhoods_data = nyc_data['features']

Let's take a look at the first item in this list.

In [5]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

#### Transform the data into a *pandas* dataframe

The next task is essentially transforming this data of nested Python dictionaries into a *pandas* dataframe. So let's start by creating an empty dataframe.

In [6]:
# define the dataframe columns
column_names = ['borough', 'neighborhood', 'latitude', 'longitude'] 

# instantiate the dataframe
nyc_data = pd.DataFrame(columns=column_names)

Take a look at the empty dataframe to confirm that the columns are as intended.

In [7]:
nyc_data

Empty DataFrame
Columns: [borough, neighborhood, latitude, longitude]
Index: []

Then let's loop through the data and fill the dataframe one row at a time.

In [8]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    nyc_data = nyc_data.append({'borough': borough,
                                'neighborhood': neighborhood_name,
                                'latitude': neighborhood_lat,
                                'longitude': neighborhood_lon
                                },
                               ignore_index=True
                               )

Quickly examine the resulting dataframe.

In [207]:
print(nyc_data.shape)
nyc_data.head()

(306, 5)


borough  borough_encoded neighborhood   latitude  longitude
0   Bronx                0    Wakefield  40.894705 -73.847201
1   Bronx                0   Co-op City  40.874294 -73.829939
2   Bronx                0  Eastchester  40.887556 -73.827806
3   Bronx                0    Fieldston  40.895437 -73.905643
4   Bronx                0    Riverdale  40.890834 -73.912585

And make sure that the dataset has all 5 boroughs and 306 neighborhoods.

In [10]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(nyc_data['borough'].unique()),
        nyc_data.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


Let's encode NYC's boroughs

In [11]:
region_encoder = preprocessing.LabelEncoder()
region_encoder.fit(nyc_data['borough'])
nyc_data.insert(1, 'borough_encoded', region_encoder.transform(nyc_data['borough']))
nyc_data.head()

borough  borough_encoded neighborhood   latitude  longitude
0   Bronx                0    Wakefield  40.894705 -73.847201
1   Bronx                0   Co-op City  40.874294 -73.829939
2   Bronx                0  Eastchester  40.887556 -73.827806
3   Bronx                0    Fieldston  40.895437 -73.905643
4   Bronx                0    Riverdale  40.890834 -73.912585

#### Use geopy library to get the latitude and longitude values of New York City.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.

In [12]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geographical coordinate of New York City are 40.7127281, -74.0060152.


#### Create a map of New York with neighborhoods superimposed on top.

In [13]:
# create map of New York using latitude and longitude values
map_nyc = folium.Map(location=[latitude, longitude], zoom_start=10)

num_borough = nyc_data['borough_encoded'].unique()
color_range = list(range(0, len(num_borough)))
colors_array = [cm.tab10(x + 1) for x in color_range]
cat_colors = [colors.rgb2hex(i) for i in colors_array]

# add markers to map
for lat, lng, borough, neighborhood, borough_encoded in zip(nyc_data['latitude'], nyc_data['longitude'], nyc_data['borough'], nyc_data['neighborhood'], nyc_data['borough_encoded']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=cat_colors[borough_encoded],
        fill=True,
        fill_color=cat_colors[borough_encoded],
        fill_opacity=0.7,
        parse_html=False).add_to(map_nyc)
    
map_nyc

## 2. Explore Neighborhoods in Manhattan

#### Define Foursquare Credentials and Version

In [14]:
%load_ext dotenv
# Use find_dotenv to locate the file
%dotenv passwords.env

CLIENT_ID = os.getenv('CLIENT_ID') # your Foursquare ID
CLIENT_SECRET = os.getenv('CLIENT_SECRET') # your Foursquare Secret
VERSION = os.getenv('VERSION') # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + ('LOADED' if CLIENT_ID is not None else 'NOT LOADED'))
print('CLIENT_SECRET:' + ('LOADED' if CLIENT_SECRET is not None else 'NOT LOADED'))

Your credentails:
CLIENT_ID: LOADED
CLIENT_SECRET:LOADED


#### Let's create a function to repeat the same process to all the neighborhoods

In [15]:
LIMIT = 100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        sleep(1)
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = [
        'neighborhood', 
        'neighborhood_latitude', 
        'neighborhood_longitude', 
        'venue', 
        'venue_latitude', 
        'venue_longitude', 
        'venue_category'
        ]
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *nyc_venues*.

In [16]:
# type your answer here
nyc_venues = getNearbyVenues(names=nyc_data['neighborhood'],
                             latitudes=nyc_data['latitude'],
                             longitudes=nyc_data['longitude']
                             )
nyc_venues.head()

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker

neighborhood  neighborhood_latitude  neighborhood_longitude  \
0    Wakefield              40.894705              -73.847201   
1    Wakefield              40.894705              -73.847201   
2    Wakefield              40.894705              -73.847201   
3    Wakefield              40.894705              -73.847201   
4    Wakefield              40.894705              -73.847201   

              venue  venue_latitude  venue_longitude  venue_category  
0  Lollipops Gelato       40.894123       -73.845892    Dessert Shop  
1          Rite Aid       40.896649       -73.844846        Pharmacy  
2         Walgreens       40.896528       -73.844700        Pharmacy  
3  Carvel Ice Cream       40.890487       -73.848568  Ice Cream Shop  
4           Dunkin'       40.890459       -73.849089      Donut Shop

#### Let's check the size of the resulting dataframe

In [17]:
print(nyc_venues.shape)
nyc_venues.head()

(10133, 7)


neighborhood  neighborhood_latitude  neighborhood_longitude  \
0    Wakefield              40.894705              -73.847201   
1    Wakefield              40.894705              -73.847201   
2    Wakefield              40.894705              -73.847201   
3    Wakefield              40.894705              -73.847201   
4    Wakefield              40.894705              -73.847201   

              venue  venue_latitude  venue_longitude  venue_category  
0  Lollipops Gelato       40.894123       -73.845892    Dessert Shop  
1          Rite Aid       40.896649       -73.844846        Pharmacy  
2         Walgreens       40.896528       -73.844700        Pharmacy  
3  Carvel Ice Cream       40.890487       -73.848568  Ice Cream Shop  
4           Dunkin'       40.890459       -73.849089      Donut Shop

Let's check how many venues were returned for each neighborhood

In [18]:
nyc_venues.groupby('neighborhood').count().head()

neighborhood_latitude  neighborhood_longitude  venue  \
neighborhood                                                          
Allerton                          31                      31     31   
Annadale                          11                      11     11   
Arden Heights                      5                       5      5   
Arlington                          6                       6      6   
Arrochar                          22                      22     22   

               venue_latitude  venue_longitude  venue_category  
neighborhood                                                    
Allerton                   31               31              31  
Annadale                   11               11              11  
Arden Heights               5                5               5  
Arlington                   6                6               6  
Arrochar                   22               22              22

#### Let's find out how many unique categories can be curated from all the returned venues

In [19]:
print('There are {} uniques categories.'.format(len(nyc_venues['venue_category'].unique())))

There are 430 uniques categories.


<a id='item3'></a>

## 3. Analyze Each Neighborhood

In [20]:
# one hot encoding
nyc_onehot = pd.get_dummies(nyc_venues[['venue_category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
nyc_onehot['neighborhood'] = nyc_venues['neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [nyc_onehot.columns[-1]] + list(nyc_onehot.columns[:-1])
nyc_onehot = nyc_onehot[fixed_columns]

print(nyc_onehot.shape)
nyc_onehot.head()

(10133, 431)


0                 0       0   

   Martial Arts School  Massage Studio  Mattress Store  Medical Center  \
0                    0               0               0               0   
1                    0               0               0               0   
2                    0               0               0               0   
3                    0               0               0               0   
4                    0               0               0               0   

   Medical Supply Store  Mediterranean Restaurant  Memorial Site  Men's Store  \
0                     0                         0              0            0   
1                     0                         0              0            0   
2                     0                         0              0            0   
3                     0                         0              0            0   
4                     0                         0              0            0   

   Metro Station  Mexican Restaurant  Middle Eastern Restaurant  Mini Golf  \
0              0                   0                          0          0   
1              0                   0                          0          0   
2              0                   0                          0          0   
3              0                   0                          0          0   
4              0                   0                          0          0   

   Miscellaneous Shop  Mobile Phone Shop  Modern European Restaurant  \
0                   0                  0                           0   
1                   0                  0                           0   
2                   0                  0                           0   
3                   0                  0                           0   
4                   0                  0                           0   

   Molecular Gastronomy Restaurant  Monument / Landmark  Moroccan Restaurant  \
0                                0                    0                    0   
1                                0                    0                    0   
2                                0                    0                    0   
3                                0                    0                    0   
4                                0                    0                    0   

   Motel  Motorcycle Shop  Movie Theater  Moving Target  Multiplex  Museum  \
0      0                0              0              0          0       0   
1      0                0              0              0          0       0   
2      0                0              0              0          0       0   
3      0                0              0              0          0       0   
4      0                0              0              0          0       0   

   Music School  Music Store  Music Venue  Nail Salon  Neighborhood  \
0             0            0            0           0             0   
1             0            0            0           0             0   
2             0            0            0           0             0   
3             0            0            0           0             0   
4             0            0            0           0             0   

   New American Restaurant  Newsstand  Nightclub  Nightlife Spot  Non-Profit  \
0                        0          0          0               0           0   
1                        0          0          0               0           0   
2                        0          0          0               0           0   
3                        0          0          0               0           0   
4                        0          0          0               0           0   

   Noodle House  North Indian Restaurant  Office  Opera House  Optical Shop  \
0             0                        0       0            0             0   
1             0                        0       0            0             0   
2             0                        

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [21]:
nyc_grouped = nyc_onehot.groupby('neighborhood').mean().reset_index()
print(nyc_grouped.shape)
nyc_grouped.head()

(302, 431)


hool  Massage Studio  Mattress Store  Medical Center  \
0             0.032258             0.0             0.0             0.0   
1             0.000000             0.0             0.0             0.0   
2             0.000000             0.0             0.0             0.0   
3             0.000000             0.0             0.0             0.0   
4             0.000000             0.0             0.0             0.0   

   Medical Supply Store  Mediterranean Restaurant  Memorial Site  Men's Store  \
0                   0.0                  0.000000            0.0          0.0   
1                   0.0                  0.000000            0.0          0.0   
2                   0.0                  0.000000            0.0          0.0   
3                   0.0                  0.000000            0.0          0.0   
4                   0.0                  0.045455            0.0          0.0   

   Metro Station  Mexican Restaurant  Middle Eastern Restaurant  Mini Golf  \
0            0.0                 0.0                   0.000000        0.0   
1            0.0                 0.0                   0.000000        0.0   
2            0.0                 0.0                   0.000000        0.0   
3            0.0                 0.0                   0.000000        0.0   
4            0.0                 0.0                   0.045455        0.0   

   Miscellaneous Shop  Mobile Phone Shop  Modern European Restaurant  \
0                 0.0                0.0                         0.0   
1                 0.0                0.0                         0.0   
2                 0.0                0.0                         0.0   
3                 0.0                0.0                         0.0   
4                 0.0                0.0                         0.0   

   Molecular Gastronomy Restaurant  Monument / Landmark  Moroccan Restaurant  \
0                              0.0                  0.0                  0.0   
1                              0.0                  0.0                  0.0   
2                              0.0                  0.0                  0.0   
3                              0.0                  0.0                  0.0   
4                              0.0                  0.0                  0.0   

   Motel  Motorcycle Shop  Movie Theater  Moving Target  Multiplex  Museum  \
0    0.0              0.0            0.0            0.0        0.0     0.0   
1    0.0              0.0            0.0            0.0        0.0     0.0   
2    0.0              0.0            0.0            0.0        0.0     0.0   
3    0.0              0.0            0.0            0.0        0.0     0.0   
4    0.0              0.0            0.0            0.0        0.0     0.0   

   Music School  Music Store  Music Venue  Nail Salon  Neighborhood  \
0           0.0          0.0          0.0         0.0           0.0   
1           0.0          0.0          0.0         0.0           0.0   
2           0.0          0.0          0.0         0.0           0.0   
3           0.0          0.0          0.0         0.0           0.0   
4           0.0          0.0          0.0         0.0           0.0   

   New American Restaurant  Newsstand  Nightclub  Nightlife Spot  Non-Profit  \
0                      0.0        0.0        0.0             0.0         0.0   
1                      0.0        0.0        0.0             0.0         0.0   
2                      0.0        0.0        0.0             0.0         0.0   
3                      0.0        0.0        0.0             0.0         0.0   
4                      0.0        0.0        0.0             0.0         0.0   

   Noodle House  North Indian Restaurant  Office  Opera House  Optical Shop  \
0           0.0                      0.0     0.0          0.0           0.0   
1           0.0                      0.0     0.0          0.0           0.0   
2           0.0                      0.0     0.0          0.0           0.0   
3         

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [23]:
def top_venues_neighboorhood(num_venues, city_grouped):
    num_top_venues = num_venues

    indicators = ['st', 'nd', 'rd']

    # create columns according to number of top venues
    columns = ['neighborhood']
    for ind in np.arange(num_top_venues):
        try:
            columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
        except:
            columns.append('{}th Most Common Venue'.format(ind+1))

    # create a new dataframe
    neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
    neighborhoods_venues_sorted['neighborhood'] = city_grouped['neighborhood']

    for ind in np.arange(city_grouped.shape[0]):
        neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(city_grouped.iloc[ind, :], num_top_venues)
    
    return neighborhoods_venues_sorted

In [24]:
neighborhoods_venues_sorted = top_venues_neighboorhood(10, nyc_grouped)

print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted.head()

(302, 11)


neighborhood 1st Most Common Venue 2nd Most Common Venue  \
0       Allerton           Pizza Place         Deli / Bodega   
1       Annadale           Pizza Place   American Restaurant   
2  Arden Heights         Deli / Bodega              Pharmacy   
3      Arlington         Deli / Bodega           Coffee Shop   
4       Arrochar              Bus Stop         Deli / Bodega   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0        Discount Store           Supermarket                   Spa   
1                  Food         Train Station            Restaurant   
2           Coffee Shop              Bus Stop           Pizza Place   
3              Bus Stop          Home Service         Boat or Ferry   
4    Italian Restaurant                 Beach                 Hotel   

       6th Most Common Venue  7th Most Common Venue 8th Most Common Venue  \
0         Chinese Restaurant            Gas Station            Playground   
1               Dance Studio                   Park         Deli / Bodega   
2          Fish & Chips Shop                Factory    Falafel Restaurant   
3              Grocery Store            Flea Market    Falafel Restaurant   
4  Middle Eastern Restaurant  Outdoors & Recreation              Pharmacy   

  9th Most Common Venue 10th Most Common Venue  
0                Bakery               Pharmacy  
1                 Diner                   Farm  
2                  Farm         Farmers Market  
3                  Farm         Farmers Market  
4          Liquor Store             Bagel Shop

<a id='item4'></a>

## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters because there are 5 boroughs in NYC.

In [25]:
nyc_grouped.head()

hool  Massage Studio  Mattress Store  Medical Center  \
0             0.032258             0.0             0.0             0.0   
1             0.000000             0.0             0.0             0.0   
2             0.000000             0.0             0.0             0.0   
3             0.000000             0.0             0.0             0.0   
4             0.000000             0.0             0.0             0.0   

   Medical Supply Store  Mediterranean Restaurant  Memorial Site  Men's Store  \
0                   0.0                  0.000000            0.0          0.0   
1                   0.0                  0.000000            0.0          0.0   
2                   0.0                  0.000000            0.0          0.0   
3                   0.0                  0.000000            0.0          0.0   
4                   0.0                  0.045455            0.0          0.0   

   Metro Station  Mexican Restaurant  Middle Eastern Restaurant  Mini Golf  \
0            0.0                 0.0                   0.000000        0.0   
1            0.0                 0.0                   0.000000        0.0   
2            0.0                 0.0                   0.000000        0.0   
3            0.0                 0.0                   0.000000        0.0   
4            0.0                 0.0                   0.045455        0.0   

   Miscellaneous Shop  Mobile Phone Shop  Modern European Restaurant  \
0                 0.0                0.0                         0.0   
1                 0.0                0.0                         0.0   
2                 0.0                0.0                         0.0   
3                 0.0                0.0                         0.0   
4                 0.0                0.0                         0.0   

   Molecular Gastronomy Restaurant  Monument / Landmark  Moroccan Restaurant  \
0                              0.0                  0.0                  0.0   
1                              0.0                  0.0                  0.0   
2                              0.0                  0.0                  0.0   
3                              0.0                  0.0                  0.0   
4                              0.0                  0.0                  0.0   

   Motel  Motorcycle Shop  Movie Theater  Moving Target  Multiplex  Museum  \
0    0.0              0.0            0.0            0.0        0.0     0.0   
1    0.0              0.0            0.0            0.0        0.0     0.0   
2    0.0              0.0            0.0            0.0        0.0     0.0   
3    0.0              0.0            0.0            0.0        0.0     0.0   
4    0.0              0.0            0.0            0.0        0.0     0.0   

   Music School  Music Store  Music Venue  Nail Salon  Neighborhood  \
0           0.0          0.0          0.0         0.0           0.0   
1           0.0          0.0          0.0         0.0           0.0   
2           0.0          0.0          0.0         0.0           0.0   
3           0.0          0.0          0.0         0.0           0.0   
4           0.0          0.0          0.0         0.0           0.0   

   New American Restaurant  Newsstand  Nightclub  Nightlife Spot  Non-Profit  \
0                      0.0        0.0        0.0             0.0         0.0   
1                      0.0        0.0        0.0             0.0         0.0   
2                      0.0        0.0        0.0             0.0         0.0   
3                      0.0        0.0        0.0             0.0         0.0   
4                      0.0        0.0        0.0             0.0         0.0   

   Noodle House  North Indian Restaurant  Office  Opera House  Optical Shop  \
0           0.0                      0.0     0.0          0.0           0.0   
1           0.0                      0.0     0.0          0.0           0.0   
2           0.0                      0.0     0.0          0.0           0.0   
3         

In [26]:
# set number of clusters
kclusters = 5
nyc_grouped_clustering = nyc_grouped.drop('neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(nyc_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 2, 1, 1, 1, 1, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [27]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'cluster_labels', kmeans.labels_)

nyc_merged = nyc_data.copy()

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
nyc_merged = nyc_merged.join(neighborhoods_venues_sorted.set_index('neighborhood'), on='neighborhood')

print(nyc_merged.shape)
nyc_merged.head() # check the last columns!

(306, 16)


borough  borough_encoded neighborhood   latitude  longitude  cluster_labels  \
0   Bronx                0    Wakefield  40.894705 -73.847201               2   
1   Bronx                0   Co-op City  40.874294 -73.829939               1   
2   Bronx                0  Eastchester  40.887556 -73.827806               2   
3   Bronx                0    Fieldston  40.895437 -73.905643               1   
4   Bronx                0    Riverdale  40.890834 -73.912585               1   

  1st Most Common Venue 2nd Most Common Venue 3rd Most Common Venue  \
0              Pharmacy           Gas Station           Pizza Place   
1           Bus Station     Accessories Store           Pizza Place   
2           Bus Station  Caribbean Restaurant         Deli / Bodega   
3                 River                 Plaza  Medical Supply Store   
4                  Park            Playground                   Gym   

  4th Most Common Venue 5th Most Common Venue 6th Most Common Venue  \
0        Sandwich Place            Laundromat        Ice Cream Shop   
1         Grocery Store  Fast Food Restaurant    Chinese Restaurant   
2                 Diner     Convenience Store  Fast Food Restaurant   
3           Yoga Studio     Fish & Chips Shop               Exhibit   
4           Bus Station                 Plaza                  Bank   

  7th Most Common Venue 8th Most Common Venue 9th Most Common Venue  \
0            Donut Shop          Dessert Shop         Deli / Bodega   
1                  Park                   Bar            Bagel Shop   
2    Seafood Restaurant    Chinese Restaurant       Automotive Shop   
3            Eye Doctor               Factory    Falafel Restaurant   
4            Food Truck          Home Service         Moving Target   

  10th Most Common Venue  
0               Fountain  
1                  Trail  
2          Bowling Alley  
3                   Farm  
4                  Field

Finally, let's visualize the resulting clusters

In [28]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
color_range = list(range(0, kclusters))
colors_array = [cm.tab10(x) for x in color_range]
cat_colors = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(nyc_merged['latitude'], nyc_merged['longitude'], nyc_merged['neighborhood'], nyc_merged['cluster_labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=cat_colors[cluster],
        fill=True,
        fill_color=cat_colors[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [29]:
nyc_merged.loc[nyc_merged['cluster_labels'] == 0, nyc_merged.columns[[2] + list(range(6, nyc_merged.shape[1]))]]

neighborhood 1st Most Common Venue 2nd Most Common Venue  \
172  Breezy Point                 Beach   Monument / Landmark   
179      Neponsit                 Beach           Yoga Studio   
204   South Beach                 Beach                  Pier   
302       Hammels                 Beach         Deli / Bodega   

    3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
172                 Trail              Bus Stop           Flower Shop   
179           Flea Market               Factory    Falafel Restaurant   
204         Deli / Bodega    Athletics & Sports            Eye Doctor   
302   Fried Chicken Joint  Fast Food Restaurant               Dog Run   

    6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
172                  Farm        Farmers Market  Fast Food Restaurant   
179                  Farm        Farmers Market  Fast Food Restaurant   
204    Falafel Restaurant                  Farm        Farmers Market   
302            Food Truck            Shoe Store  Gym / Fitness Center   

    9th Most Common Venue 10th Most Common Venue  
172                 Field    Filipino Restaurant  
179                 Field    Filipino Restaurant  
204  Fast Food Restaurant                  Field  
302                 Diner               Bus Stop

#### Cluster 2

In [30]:
nyc_merged.loc[nyc_merged['cluster_labels'] == 1, nyc_merged.columns[[2] + list(range(6, nyc_merged.shape[1]))]]

e   
57                          Bakery                 Lingerie Store   
58                Sushi Restaurant                            Bar   
59                     Pizza Place                    Coffee Shop   
60                  Farmers Market                     Playground   
61                             Gym      Middle Eastern Restaurant   
62                        Pharmacy  Vegetarian / Vegan Restaurant   
63             Fried Chicken Joint                      Gift Shop   
64                       Pet Store                            Bar   
65                             Bar                         Bakery   
66                             Bar                   Gourmet Shop   
67                     Flower Shop                    Pizza Place   
68                     Art Gallery                     Playground   
69                     Pizza Place                     Playground   
70                       Bookstore             Mexican Restaurant   
71                     Supermarket                           Food   
74                            Farm                 Farmers Market   
75   Paper / Office Supplies Store                         Lounge   
77                           Beach                           Farm   
79                 Bubble Tea Shop           Cantonese Restaurant   
81                            Farm                 Farmers Market   
82              Seafood Restaurant              Convenience Store   
84               Indian Restaurant             Chinese Restaurant   
85                     Fish Market             Falafel Restaurant   
86                          Bakery                   Cocktail Bar   
87          Furniture / Home Store                            Spa   
88                     Supermarket              Indian Restaurant   
90            Fast Food Restaurant                           Bank   
91                         Factory             Falafel Restaurant   
93              Mexican Restaurant                     Donut Shop   
94              Miscellaneous Shop                 Sandwich Place   
95                  Sandwich Place                    Music Venue   
96                    Burger Joint                       Dive Bar   
97                    Burger Joint              Korean Restaurant   
99                             Pub                     Donut Shop   
100                            Spa                   Optical Shop   
101                    Coffee Shop                           Bank   
102             Chinese Restaurant                         Bakery   
103             Chinese Restaurant               Sushi Restaurant   
104                         Lounge                     Bike Trail   
105                        Library           Ethiopian Restaurant   
106                  Grocery Store          Performing Arts Venue   
107                      Juice Bar              French Restaurant   
108                      Wine Shop                          Diner   
109           Gym / Fitness Center                            Gym   
110               Greek Restaurant                           Park   
111                    Coffee Shop                 Ice Cream Shop   
112                         Bakery                      Wine Shop   
113                   Cocktail Bar                          Hotel   
114                           Café                    Pizza Place   
115           Gym / Fitness Center                 Sandwich Place   
116                        Theater              French Restaurant   
117                Bubble Tea Shop                   Burger Joint   
118  Vegetarian / Vegan Restaurant              Korean Restaurant   
119                    Pizza Place                       Pharmacy   
120                            Spa                     Skate Park   
121                    Pizza Place       Mediterranean Restaurant   
122       Mediterranean Restaurant            Sporting Goods Shop   
123                    Coffee Shop                        Thea

#### Cluster 3

In [31]:
nyc_merged.loc[nyc_merged['cluster_labels'] == 2, nyc_merged.columns[[2] + list(range(6, nyc_merged.shape[1]))]]

South American Restaurant                       Park   
137                Deli / Bodega                 Donut Shop   
140               Discount Store              Grocery Store   
142                Grocery Store         Chinese Restaurant   
144          Arts & Crafts Store                Flea Market   
146             Department Store                Pizza Place   
149                       Bakery                 Steakhouse   
150                  Flower Shop         Falafel Restaurant   
153         Cantonese Restaurant                     Bakery   
154             Sushi Restaurant                Supermarket   
156                        Motel                Pizza Place   
157                 Dance Studio                 Restaurant   
158                  Pizza Place              Grocery Store   
159            Indian Restaurant               Home Service   
161                   Donut Shop                   Pharmacy   
169         Caribbean Restaurant         Chinese Restaurant   
170                         Bank              Grocery Store   
171                  Pizza Place                Flea Market   
173                 Dessert Shop         Italian Restaurant   
174                          Gym                   Boutique   
176                         Park         Seafood Restaurant   
181                Grocery Store         Salon / Barbershop   
187      Fruit & Vegetable Store                        Gym   
190           Italian Restaurant         Mexican Restaurant   
191            French Restaurant                       Bank   
193                      Factory         Falafel Restaurant   
194                          Gym                IT Services   
195         Gym / Fitness Center                     Museum   
198               Discount Store              Bowling Alley   
202           Falafel Restaurant                       Farm   
205                   Donut Shop                Fish Market   
206                     Bus Stop        Filipino Restaurant   
208                Go Kart Track                 Bagel Shop   
209                Deli / Bodega               Optical Shop   
211                   Bagel Shop                 Hobby Shop   
214               Sandwich Place       Fast Food Restaurant   
215                   Restaurant               Dance Studio   
217       Thrift / Vintage Store                   Bus Stop   
218                  Supermarket                       Café   
220               Discount Store              Grocery Store   
221               Clothing Store                 Food Truck   
227                Boat or Ferry              Grocery Store   
228                        Hotel  Middle Eastern Restaurant   
229          Japanese Restaurant                 Restaurant   
231                     Pharmacy                 Bagel Shop   
232           Chinese Restaurant                  Pet Store   
234               Scenic Lookout                       Food   
237                 Liquor Store       Fast Food Restaurant   
240                   Donut Shop             Ice Cream Shop   
241                  Pizza Place          Fish & Chips Shop   
243                   Restaurant                Coffee Shop   
246                Deli / Bodega              Grocery Store   
252                          Bar                  Gastropub   
253                  Pizza Place               Burger Joint   
254                   Bagel Shop       Gym / Fitness Center   
256                  Yoga Studio                Fish Market   
258               Ice Cream Shop                       Café   
260           Salon / Barbershop                Pizza Place   
262                   Bagel Shop         Italian Restaurant   
264               History Museum                Pizza Place   
267           Chinese Restaurant               Liquor Store   
268                     Pharmacy         Mexican Restaurant   
269           Spanish Restaurant                Supermarket   
270             Video Game Store      

#### Cluster 4

In [32]:
nyc_merged.loc[nyc_merged['cluster_labels'] == 3, nyc_merged.columns[[2] + list(range(6, nyc_merged.shape[1]))]]

neighborhood 1st Most Common Venue 2nd Most Common Venue  \
192   Somerville                  Park           Yoga Studio   
203    Todt Hill                  Park           Yoga Studio   

    3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
192               Exhibit               Factory    Falafel Restaurant   
203               Exhibit               Factory    Falafel Restaurant   

    6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
192                  Farm        Farmers Market  Fast Food Restaurant   
203                  Farm        Farmers Market  Fast Food Restaurant   

    9th Most Common Venue 10th Most Common Venue  
192                 Field    Filipino Restaurant  
203                 Field    Filipino Restaurant

#### Cluster 5

In [33]:
nyc_merged.loc[nyc_merged['cluster_labels'] == 4, nyc_merged.columns[[2] + list(range(6, nyc_merged.shape[1]))]]

neighborhood 1st Most Common Venue 2nd Most Common Venue  \
76    Mill Island                  Pool           Flea Market   
238  Butler Manor                  Pool        Baseball Field   

    3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
76             Eye Doctor               Factory    Falafel Restaurant   
238           Flea Market            Eye Doctor               Factory   

    6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
76                   Farm        Farmers Market  Fast Food Restaurant   
238    Falafel Restaurant                  Farm        Farmers Market   

    9th Most Common Venue 10th Most Common Venue  
76                  Field    Filipino Restaurant  
238  Fast Food Restaurant                  Field

# London

London is composed of 32 boroughs plus the City of London that are grouped in 5 sub-regions. Each borough is composed of a few "areas" which are equivalent to the neighborhoods in New York City. So in order to create a similar dataset I will collect data from two wikipedia pages and combine the results.

In [34]:
url_areas = "https://en.wikipedia.org/wiki/List_of_areas_of_London"
url_regions = 'https://en.wikipedia.org/wiki/List_of_sub-regions_used_in_the_London_Plan'

session = requests.session()
session.verify = False

resp_areas = session.get(url_areas)
resp_regions = session.get(url_regions)

root_areas = html.fromstring(resp_areas.content)
root_regions = html.fromstring(resp_regions.content)

In [35]:
# Function to extract data from wikipedia tables
def extract_data(root):
    row_combox = root.xpath('.//table[contains(@class, "wikitable")]')
    row_combox = row_combox[0].xpath('.//tbody/tr')
    cols = [unidecode(line.text.strip()).replace(" ", "_").lower() if line.text is not None else None for line in list(row_combox[0])]

    data = []
    for row in row_combox[1:]:
        row_dict = {}
        for col in zip(cols, list(row)):
            row_dict[col[0]] = col[1].text_content().split('[')[0].strip()
        
        data.append(row_dict)
    
    return data

In [36]:
london_areas = extract_data(root_areas)
london_areas = pd.DataFrame(london_areas)

print(london_areas.shape)
london_areas.head()

(533, 6)


location                  london_borough       post_town  \
0   Abbey Wood              Bexley,  Greenwich          LONDON   
1        Acton  Ealing, Hammersmith and Fulham          LONDON   
2    Addington                         Croydon         CROYDON   
3   Addiscombe                         Croydon         CROYDON   
4  Albany Park                          Bexley  BEXLEY, SIDCUP   

  postcode_district dial_code os_grid_ref  
0               SE2       020    TQ465785  
1            W3, W4       020    TQ205805  
2               CR0       020    TQ375645  
3               CR0       020    TQ345665  
4         DA5, DA14       020    TQ478728

### Data Wrangling 

First, I will modify the london_areas data so there is only one borough per location. The reason for that, is because in order to assign a region to a location, I need to know its borough. Let's use the geopy library to search the location and use the borough it returns. 

Remove parentheses from location  

In [37]:
def remove_parentheses(row):
    location = row['location']
    if '(' in location:
        location = location.split('(')[0].strip()
    return location

In [38]:
london_areas['location'] = london_areas.apply(remove_parentheses, axis=1)

print(london_areas.shape)
london_areas.head()

(533, 6)


location                  london_borough       post_town  \
0   Abbey Wood              Bexley,  Greenwich          LONDON   
1        Acton  Ealing, Hammersmith and Fulham          LONDON   
2    Addington                         Croydon         CROYDON   
3   Addiscombe                         Croydon         CROYDON   
4  Albany Park                          Bexley  BEXLEY, SIDCUP   

  postcode_district dial_code os_grid_ref  
0               SE2       020    TQ465785  
1            W3, W4       020    TQ205805  
2               CR0       020    TQ375645  
3               CR0       020    TQ345665  
4         DA5, DA14       020    TQ478728

Select locations with more than one borough

In [39]:
borough_comma = london_areas.loc[london_areas['london_borough'].str.contains(', | &')]

print(borough_comma.shape)
borough_comma.head()

(27, 6)


location                            london_borough post_town  \
0        Abbey Wood                        Bexley,  Greenwich    LONDON   
1             Acton            Ealing, Hammersmith and Fulham    LONDON   
85   Chadwell Heath           Redbridge, Barking and Dagenham   ROMFORD   
99         Chiswick  Hounslow, Ealing, Hammersmith and Fulham    LONDON   
102         Clapham                       Lambeth, Wandsworth    LONDON   

    postcode_district dial_code os_grid_ref  
0                 SE2       020    TQ465785  
1              W3, W4       020    TQ205805  
85                RM6       020    TQ485885  
99                 W4       020    TQ205785  
102               SW4       020    TQ295755

In [40]:
def select_neighborhood(row):
    area = row['location']
    borough = row['london_borough']
    nomi = Nominatim(user_agent='london_app')
    result = nomi.geocode(f'{area}, London, United Kingdom')
    
    for line in result[0].split(','):
        if 'Borough of' in line:
            borough = line.split('of')[-1].strip()
            break
        
    return borough

In [41]:
borough_comma.loc[:, 'london_borough'] = borough_comma.apply(select_neighborhood, axis=1)
borough_comma.loc[borough_comma['london_borough'].str.contains(', | &')]

location     london_borough post_town postcode_district dial_code  \
457   Temple  City, Westminster    LONDON          EC4, WC2       020   

    os_grid_ref  
457    TQ311809

As we can see there are still some locations where it wasn't possible to update the neighborhood using geopy so those I will research online and fix by hand.  

In [42]:
indexes = borough_comma.loc[borough_comma['london_borough'].str.contains(', | &')].index
new_boroughs = ['City']
borough_comma.loc[indexes, 'london_borough'] = new_boroughs

print(borough_comma.shape)
borough_comma

(27, 6)


location london_borough                        post_town  \
0        Abbey Wood      Greenwich                           LONDON   
1             Acton         Ealing                           LONDON   
85   Chadwell Heath      Redbridge                          ROMFORD   
99         Chiswick       Hounslow                           LONDON   
102         Clapham     Wandsworth                           LONDON   
104     Cockfosters        Enfield                           BARNET   
121     Cricklewood          Brent                           LONDON   
168      Falconwood         Bexley                  LONDON, WELLING   
169      Farringdon      Islington                           LONDON   
173   Finsbury Park      Islington                           LONDON   
249       Horn Park      Greenwich                           LONDON   
260      Kennington      Southwark                           LONDON   
264          Kenton          Brent                   HARROW, LONDON   
268         Kilburn          Brent                           LONDON   
358      Park Royal         Ealing                           LONDON   
379      Queensbury          Brent  HARROW, STANMOREEDGWARE, LONDON   
392          Ruxley         Bexley                SIDCUP, ORPINGTON   
409     Snaresbrook      Redbridge                           LONDON   
448         Sudbury         Newham                  HARROW, WEMBLEY   
454        Sydenham       Lewisham                           LONDON   
455   Sydenham Hill      Southwark                           LONDON   
457          Temple           City                           LONDON   
459      Thamesmead      Greenwich                    LONDON, ERITH   
470    Tufnell Park         Camden                           LONDON   
479    Upper Ruxley         Bexley                SIDCUP, ORPINGTON   
524  Woodford Green      Redbridge           LONDON, WOODFORD GREEN   
529  Worcester Park         Sutton                   WORCESTER PARK   

    postcode_district   dial_code os_grid_ref  
0                 SE2         020    TQ465785  
1              W3, W4         020    TQ205805  
85                RM6         020    TQ485885  
99                 W4         020    TQ205785  
102               SW4         020    TQ295755  
104               EN4         020    TQ275965  
121               NW2         020    TQ235855  
168         SE9, DA16         020    TQ455755  
169               EC1         020    TQ315818  
173                N4         020    TQ314872  
249              SE12         020    TQ408739  
260              SE11         020    TQ305775  
264               HA3         020    TQ175885  
268               NW6         020    TQ245835  
358              NW10         020    TQ195828  
379     HA3, HA7, HA8         020    TQ185895  
392         DA14, BR5  020, 01689    TQ485704  
409               E11         020    TQ395895  
448          HA0, HA1         020    TQ165852  
454              SE26         020    TQ352714  
455        SE21, SE26         020    TQ352714  
457          EC4, WC2         020    TQ311809  
459   SE28, SE2, DA18         020    TQ475805  
470           N7, N19         020    TQ295855  
479         DA14, BR5  020, 01689      TQ4970  
524               IG8         020    TQ405915  
529               KT4         020    TQ225655

In [43]:
london_areas.loc[borough_comma.index, 'london_borough'] = borough_comma['london_borough']

print(london_areas.iloc[borough_comma.index].shape)
london_areas.iloc[borough_comma.index].head()

(27, 6)


location london_borough post_town postcode_district dial_code  \
0        Abbey Wood      Greenwich    LONDON               SE2       020   
1             Acton         Ealing    LONDON            W3, W4       020   
85   Chadwell Heath      Redbridge   ROMFORD               RM6       020   
99         Chiswick       Hounslow    LONDON                W4       020   
102         Clapham     Wandsworth    LONDON               SW4       020   

    os_grid_ref  
0      TQ465785  
1      TQ205805  
85     TQ485885  
99     TQ205785  
102    TQ295755

Convert OS Grid (British National Grid References) to Latitude & Longitude

In [44]:
from OSGridConverter import grid2latlong

def grid_conversion(row):
    grid = row['os_grid_ref'].strip()
    latitude = None
    longitude = None
    if len(grid):
        coord = grid2latlong(grid)
        latitude = coord.latitude
        longitude = coord.longitude
    
    else:
        nomi = Nominatim(user_agent='london_app')
        location = row['location']
        borough = row['london_borough']
        result = nomi.geocode(f'{location}, {borough}, London, United Kingdom')

        if result is not None:
            latitude = result[1][0]
            longitude = result[1][1]

    return latitude, longitude

In [45]:
london_areas[['latitude', 'longitude']] = ''
london_areas[['latitude', 'longitude']] = london_areas.apply(grid_conversion, axis=1, result_type='expand')

print(london_areas.shape)
london_areas.head()

(533, 8)


location london_borough       post_town postcode_district dial_code  \
0   Abbey Wood      Greenwich          LONDON               SE2       020   
1        Acton         Ealing          LONDON            W3, W4       020   
2    Addington        Croydon         CROYDON               CR0       020   
3   Addiscombe        Croydon         CROYDON               CR0       020   
4  Albany Park         Bexley  BEXLEY, SIDCUP         DA5, DA14       020   

  os_grid_ref   latitude  longitude  
0    TQ465785  51.486484   0.109318  
1    TQ205805  51.510591  -0.264585  
2    TQ375645  51.362934  -0.025780  
3    TQ345665  51.381625  -0.068126  
4    TQ478728  51.434929   0.125663

Rename borough "City" to "City of London"

In [46]:
london_areas['london_borough'] = london_areas['london_borough'].str.replace('City', 'City of London')
london_areas.head(10)

location  london_borough       post_town postcode_district  \
0        Abbey Wood       Greenwich          LONDON               SE2   
1             Acton          Ealing          LONDON            W3, W4   
2         Addington         Croydon         CROYDON               CR0   
3        Addiscombe         Croydon         CROYDON               CR0   
4       Albany Park          Bexley  BEXLEY, SIDCUP         DA5, DA14   
5  Aldborough Hatch       Redbridge          ILFORD               IG2   
6           Aldgate  City of London          LONDON               EC3   
7           Aldwych     Westminster          LONDON               WC2   
8          Alperton           Brent         WEMBLEY               HA0   
9           Anerley         Bromley          LONDON              SE20   

  dial_code os_grid_ref   latitude  longitude  
0       020    TQ465785  51.486484   0.109318  
1       020    TQ205805  51.510591  -0.264585  
2       020    TQ375645  51.362934  -0.025780  
3       020    TQ345665  51.381625  -0.068126  
4       020    TQ478728  51.434929   0.125663  
5       020    TQ455895  51.585581   0.099459  
6       020    TQ334813  51.514885  -0.078356  
7       020    TQ307810  51.512819  -0.117388  
8       020    TQ185835  51.537976  -0.292401  
9       020    TQ345695  51.408585  -0.066989

Fix problematic locations

In [47]:
london_areas.loc[(london_areas['location'] == 'Brompton') | (london_areas['location'] == 'Sudbury') | (london_areas['location'] == 'Somerstown')]

location                                london_borough  \
68     Brompton  Kensington and ChelseaHammersmith and Fulham   
411  Somerstown                                        Camden   
448     Sudbury                                        Newham   

           post_town postcode_district dial_code os_grid_ref   latitude  \
68            LONDON               SW3       020    TQ275795  51.500069   
411           LONDON               NW1       020    TQ295825  51.526575   
448  HARROW, WEMBLEY          HA0, HA1       020    TQ165852  51.553670   

     longitude  
68   -0.164056  
411  -0.134133  
448  -0.320682

In [48]:
london_areas.loc[68, 'london_borough'] = 'Kensington and Chelsea'
london_areas.loc[411, 'location'] = 'Somers Town'
london_areas.loc[448, 'london_borough'] = 'Brent'
london_areas.iloc[[68, 411, 448]]

location          london_borough        post_town postcode_district  \
68      Brompton  Kensington and Chelsea           LONDON               SW3   
411  Somers Town                  Camden           LONDON               NW1   
448      Sudbury                   Brent  HARROW, WEMBLEY          HA0, HA1   

    dial_code os_grid_ref   latitude  longitude  
68        020    TQ275795  51.500069  -0.164056  
411       020    TQ295825  51.526575  -0.134133  
448       020    TQ165852  51.553670  -0.320682

Now that we fixed the dataset, let's check if we got 33 unique boroughs, which is London's official number of boroughs.

In [49]:
len(london_areas['london_borough'].unique())

36

We have 3 more boroughs than we should have. In order to find out which ones are wrong, I will download an oficial government dataset containing information about London's boroughs and will compare it with our list.

In [50]:
official_boroughs = pd.read_csv('https://data.london.gov.uk/download/london-borough-profiles/c1693b82-68b1-44ee-beb2-3decf17dc1f8/london-borough-profiles.csv', engine='python', encoding='latin1')
official_boroughs.head()

India      

  %_of_largest_migrant_population_(2011)  \
0                                    2.8   
1                                    4.7   
2                                    3.1   
3                                    2.6   
4                                    9.2   

  Second_largest_migrant_population_by_country_of_birth_(2011)  \
0                                             France             
1                                              India             
2                                             Poland             
3                                              India             
4                                             Poland             

  %_of_second_largest_migrant_population_(2011)  \
0                                             2   
1                                           2.3   
2                                           2.4   
3                                           1.5   
4                                           3.4   

  Third_largest_migrant_population_by_country_of_birth_(2011)  \
0                                          Australia            
1                                           Pakistan            
2                                               Iran            
3                                            Ireland            
4                                            Ireland            

  %_of_third_largest_migrant_population_(2011)  \
0                                          1.9   
1                                          2.3   
2                                            2   
3                                          0.9   
4                                          2.9   

  %_of_population_from_BAME_groups_(2016)  \
0                                    27.5   
1                                    49.5   
2                                    38.7   
3                                    21.4   
4                                    64.9   

  %_people_aged_3+_whose_main_language_is_not_English_(2011_Census)  \
0                                               17.1                  
1                                               18.7                  
2                                               23.4                  
3                                                  6                  
4                                               37.2                  

  Overseas_nationals_entering_the_UK_(NINo),_(2015/16)  \
0                                                975     
1                                              7,538     
2                                             13,094     
3                                              2,198     
4                                             22,162     

   New_migrant_(NINo)_rates,_(2015/16)  \
0                                152.2   
1                                 59.1   
2                                 53.1   
3                                 14.4   
4                                100.9   

  Largest_migrant_population_arrived_during_2015/16  \
0                                             India   
1                                           Romania   
2                                           Romania   
3                                           Romania   
4                                           Romania   

  Second_largest_migrant_population_arrived_during_2015/16  \
0                                             France         
1                                           Bulgaria         
2                                             Poland         
3                                             Poland         
4                                              Italy         

  Third_largest_migrant_population_arrived_during_2015/16  \
0                                      United States        
1                                          Lithuania        
2                                              Italy        
3                                            Nigeria        
4                

In [51]:
official_boroughs.dropna(inplace=True)
official_areas = official_boroughs['Area_name']
official_areas.head()

1    Barking and Dagenham
2                  Barnet
3                  Bexley
4                   Brent
5                 Bromley
Name: Area_name, dtype: object

In [52]:
# Extra boroughs
myareas = pd.Series(london_areas['london_borough'].unique())

myareas.loc[~myareas.isin(official_areas)]

5           City of London
32                Dartford
34    Camden and Islington
35     Haringey and Barnet
dtype: object

Fix problematic boroughs

In [53]:
london_areas.loc[(london_areas['london_borough'] == 'Dartford')]

location london_borough post_town postcode_district dial_code  \
133  Dartford       Dartford    LONDON               DA5       020   

    os_grid_ref  latitude  longitude  
133    TQ287861  51.55911  -0.144352

In [54]:
london_areas.loc[133, 'london_borough'] = 'Bexley'
london_areas['london_borough'] = london_areas['london_borough'].str.replace('Camden and Islington', 'Camden')
london_areas['london_borough'] = london_areas['london_borough'].str.replace('Haringey and Barnet', 'Haringey')


len(london_areas['london_borough'].unique())

33

In [55]:
# Rename columns
london_areas.rename(columns={'location': 'neighborhood', 'london_borough': 'borough'}, inplace=True)

Now let's collect the assign the locations to its respective regions 

In [56]:
london_regions = extract_data(root_regions)
london_regions = pd.DataFrame(london_regions)
print(london_regions.shape)
london_regions.head()

(5, 3)


sub-region                                    london_boroughs  \
0    Central  Camden, City of London, Kensington and Chelsea...   
1       East  Barking and Dagenham, Bexley, Greenwich, Hackn...   
2      North                          Barnet, Enfield, Haringey   
3      South  Bromley, Croydon, Kingston upon Thames, Merton...   
4       West  Brent, Ealing, Hammersmith and Fulham, Harrow,...   

                                  area  
0   13,136.02 hectares (50.7185 sq mi)  
1  47,673.77 hectares (184.0695 sq mi)  
2   19,853.67 hectares (76.6554 sq mi)  
3  44,936.52 hectares (173.5009 sq mi)  
4  33,868.62 hectares (130.7675 sq mi)

Spread the dataset

In [57]:
london_regions = pd.DataFrame(london_regions['london_boroughs'].str.split(',').tolist(), index=london_regions['sub-region']).stack()
london_regions = london_regions.reset_index()[['sub-region', 0]]

london_regions = london_regions.apply(lambda x: x.str.strip())

london_regions.rename(columns={'sub-region': 'sub_region', 0: 'borough'}, inplace=True)
london_regions.head()

sub_region                 borough
0    Central                  Camden
1    Central          City of London
2    Central  Kensington and Chelsea
3    Central               Islington
4    Central                 Lambeth

Now, let's merge the tables

In [58]:
london_data = pd.merge(london_regions, london_areas[['borough', 'neighborhood', 'latitude', 'longitude']], on='borough', how='inner').reset_index(drop=True)
london_data.head()

sub_region borough  neighborhood   latitude  longitude
0    Central  Camden  Belsize Park  51.545049  -0.165131
1    Central  Camden    Bloomsbury  51.526345  -0.119715
2    Central  Camden   Camden Town  51.544548  -0.133398
3    Central  Camden    Chalk Farm  51.543969  -0.153628
4    Central  Camden     Fitzrovia  51.518533  -0.137347

In [59]:
# Check for nulls 
london_data.isnull().sum()

sub_region      0
borough         0
neighborhood    0
latitude        0
longitude       0
dtype: int64

In [60]:
num_neighborhoods = london_data.shape[0]
num_boroughs = len(london_data['borough'].unique())
print(f'There are {num_boroughs} boroughs and {num_neighborhoods} neighborhoods in London.')

There are 33 boroughs and 533 neighborhoods in London.


Now that the dataset is clean, let's check for duplicates in the coordinates. The reason this wasn't done before is because I wanted to fix the inconsistencies beforehand so I have the correct data to search for the correct coordinates in case I have to fix something.

In [61]:
duplicated_coord = london_data.loc[london_data.duplicated(subset=['latitude', 'longitude'], keep=False)]
print(duplicated_coord.shape)
duplicated_coord.head()

(103, 5)


sub_region borough  neighborhood   latitude  longitude
1     Central  Camden    Bloomsbury  51.526345  -0.119715
8     Central  Camden      Highgate  51.571737  -0.146726
9     Central  Camden       Holborn  51.517359  -0.120085
11    Central  Camden  King's Cross  51.535100  -0.104923
14    Central  Camden      St Giles  51.517359  -0.120085

In [62]:
print(f"There are {duplicated_coord.shape[0]} neighborhoods that have potentially bad coordinates so I will use geopy to search for all of them")

There are 103 neighborhoods that have potentially bad coordinates so I will use geopy to search for all of them


In [63]:
nomi = Nominatim(user_agent='london')
nomi.geocode(f'Bloomsbury, Camden, London, United Kingdom')[1]

(51.523126, -0.1260664)

In [64]:
from time import sleep

# Get coordinates 
def get_coordinates(row):
    neighborhood = row['neighborhood'] 
    borough = row['borough']
    nomi = Nominatim(user_agent='london')
    result = nomi.geocode(f'{neighborhood}, {borough}, London, United Kingdom')
    
    latitude = None
    longitude = None
    
    if result is not None:
        latitude = result[1][0]
        longitude = result[1][1]
        
    return latitude, longitude

In [65]:
duplicated_coord[['latitude', 'longitude']] = duplicated_coord.apply(get_coordinates, axis=1, result_type='expand')
duplicated_coord

sub_region                 borough         neighborhood   latitude  \
1      Central                  Camden           Bloomsbury  51.523126   
8      Central                  Camden             Highgate  51.577683   
9      Central                  Camden              Holborn  51.517934   
11     Central                  Camden         King's Cross  51.532395   
14     Central                  Camden             St Giles  51.515472   
15     Central                  Camden           St Pancras  51.532291   
23     Central  Kensington and Chelsea             Brompton  51.486976   
27     Central  Kensington and Chelsea           Kensington  51.496884   
28     Central  Kensington and Chelsea     North Kensington  51.521151   
32     Central               Islington                Angel  51.531842   
33     Central               Islington              Archway  51.565437   
36     Central               Islington          Clerkenwell  51.523727   
39     Central               Islington             Finsbury  51.526574   
42     Central               Islington             Holloway  51.557245   
44     Central               Islington           Nag's Head  51.533850   
45     Central               Islington          Pentonville  51.532105   
46     Central               Islington            St Luke's  51.526867   
51     Central                 Lambeth              Lambeth  51.501301   
52     Central                 Lambeth                 Oval  51.483752   
56     Central                 Lambeth             Vauxhall  51.487483   
58     Central               Southwark             Bankside  51.507499   
66     Central               Southwark            Newington  51.485289   
71     Central               Southwark        Sydenham Hill  51.432604   
75     Central             Westminster            Belgravia  51.492491   
77     Central             Westminster            Chinatown  51.511680   
79     Central             Westminster        Knightsbridge  51.501755   
85     Central             Westminster             Millbank  51.492612   
91     Central             Westminster          Westminster  51.500444   
104       East                  Bexley               Bexley  51.441679   
105       East                  Bexley          Bexleyheath  51.463565   
121       East                  Bexley            North End        NaN   
125       East                  Bexley          Slade Green  51.467673   
127       East                  Bexley                Upton  51.452457   
128       East                  Bexley              Welling  51.465290   
137       East               Greenwich            Maze Hill  51.482607   
144       East               Greenwich       Westcombe Park  51.484214   
145       East               Greenwich             Woolwich  51.491953   
146       East                 Hackney              Dalston  51.546117   
147       East                 Hackney              Hackney  51.545623   
148       East                 Hackney      Hackney Central  51.547061   
149       East                 Hackney      Hackney Marshes  51.556496   
150       East                 Hackney         Hackney Wick  51.543433   
151       East                 Hackney           Haggerston  51.538518   
152       East                 Hackney             Homerton  51.547079   
153       East                 Hackney               Hoxton  51.532393   
155       East                 Hackney        Lower Clapton  51.554657   
158       East                 Hackney           Shoreditch  51.526669   
195       East                Lewisham         Hither Green  51.451587   
198       East                Lewisham                  Lee  51.449673   
203       East                Lewisham             Sydenham  51.427742   
212       East                  Newham       North Woolwich  51.500407   
216       East                  Newham           Upton Park  51.535106   
217       East                  Newham             West Ham  51.528097   
229     

Look for, and fix nulls values in the new coordinates

In [66]:
duplicated_coord.loc[duplicated_coord.isnull().any(1)]

sub_region               borough  neighborhood  latitude  longitude
121       East                Bexley     North End       NaN        NaN
255       East        Waltham Forest     Cann Hall       NaN        NaN
375      South               Croydon  Old Coulsdon       NaN        NaN
398      South  Kingston upon Thames    Old Malden       NaN        NaN

After checking on Google, the original coordinates for those two neighborhoods were correct so I will keep them in the dataset and update the new ones.

In [67]:
index2update = np.setdiff1d(duplicated_coord.index, duplicated_coord.loc[duplicated_coord.isnull().any(1)].index)
london_data.loc[index2update, ['latitude', 'longitude']] = duplicated_coord.loc[index2update, ['latitude', 'longitude']]

print(f"There are {london_data.duplicated(subset=['latitude', 'longitude'], keep=False).sum()} duplicate coordinates left.")

There are 0 duplicate coordinates left.


#### Change the coordinates columns data type

In [68]:
london_data['latitude'] = london_data['latitude'].astype('float64')
london_data['longitude'] = london_data['longitude'].astype('float64')
london_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533 entries, 0 to 532
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sub_region    533 non-null    object 
 1   borough       533 non-null    object 
 2   neighborhood  533 non-null    object 
 3   latitude      533 non-null    float64
 4   longitude     533 non-null    float64
dtypes: float64(2), object(3)
memory usage: 20.9+ KB


Use geopy library to get the latitude and longitude values of London.

In [69]:
address = 'London, England'

geolocator = Nominatim(user_agent="london_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of London are {}, {}.'.format(latitude, longitude))

The geographical coordinate of London are 51.5073219, -0.1276474.


Encode "sub_region" column

In [70]:
from sklearn import preprocessing

region_encoder = preprocessing.LabelEncoder()
region_encoder.fit(london_data['sub_region'])
region_encoder.classes_

array(['Central', 'East', 'North', 'South', 'West'], dtype=object)

In [71]:
london_data.insert(1, 'sub_region_encoded', region_encoder.transform(london_data['sub_region']))
london_data.head()

sub_region  sub_region_encoded borough  neighborhood   latitude  longitude
0    Central                   0  Camden  Belsize Park  51.545049  -0.165131
1    Central                   0  Camden    Bloomsbury  51.523126  -0.126066
2    Central                   0  Camden   Camden Town  51.544548  -0.133398
3    Central                   0  Camden    Chalk Farm  51.543969  -0.153628
4    Central                   0  Camden     Fitzrovia  51.518533  -0.137347

#### Create a map of London with neighborhoods superimposed on top.

In [72]:
# create map of London using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the sub regions
num_regions = london_data['sub_region_encoded'].unique()
color_range = list(range(0, len(num_regions)))
colors_array = [cm.tab10(x + 1) for x in color_range]
cat_colors = [colors.rgb2hex(i) for i in colors_array]

# add markers to map
for lat, lng, borough, neighborhood, region in zip(london_data['latitude'], london_data['longitude'], london_data['borough'], london_data['neighborhood'], london_data['sub_region_encoded']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=cat_colors[region],
        fill=True,
        fill_color=cat_colors[region],
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

## 2. Explore Neighborhoods in London

In [73]:
london_venues = getNearbyVenues(names=london_data['neighborhood'],
                                latitudes=london_data['latitude'],
                                longitudes=london_data['longitude']
                                )
print(london_venues.shape)
london_venues.head()

Belsize Park
Bloomsbury
Camden Town
Chalk Farm
Fitzrovia
Frognal
Gospel Oak
Hampstead
Highgate
Holborn
Kentish Town
King's Cross
Primrose Hill
Somers Town
St Giles
St Pancras
Swiss Cottage
Tufnell Park
West Hampstead
Aldgate
Barbican
Blackfriars
Temple
Brompton
Chelsea
Earls Court
Holland Park
Kensington
North Kensington
Notting Hill
South Kensington
West Brompton
Angel
Archway
Barnsbury
Canonbury
Clerkenwell
De Beauvoir Town
Farringdon
Finsbury
Finsbury Park
Highbury
Holloway
Islington
Nag's Head
Pentonville
St Luke's
Upper Holloway
Brixton
Gipsy Hill
Herne Hill
Lambeth
Oval
Stockwell
Streatham
Tulse Hill
Vauxhall
West Norwood
Bankside
Bermondsey
Camberwell
Denmark Hill
Dulwich
East Dulwich
Elephant and Castle
Kennington
Newington
Nunhead
Peckham
Rotherhithe
Surrey Quays
Sydenham Hill
Walworth
Aldwych
Bayswater
Belgravia
Charing Cross
Chinatown
Covent Garden
Knightsbridge
Lisson Grove
Little Venice
Maida Vale
Marylebone
Mayfair
Millbank
Paddington
Pimlico
Soho
St James's
St John's Woo

neighborhood  neighborhood_latitude  neighborhood_longitude  \
0  Belsize Park              51.545049               -0.165131   
1  Belsize Park              51.545049               -0.165131   
2  Belsize Park              51.545049               -0.165131   
3  Belsize Park              51.545049               -0.165131   
4  Belsize Park              51.545049               -0.165131   

            venue  venue_latitude  venue_longitude venue_category  
0       Chamomile       51.545729        -0.162398           Café  
1  The Washington       51.545467        -0.162768            Pub  
2   Black Truffle       51.545977        -0.162530  Deli / Bodega  
3       Starbucks       51.545459        -0.162607    Coffee Shop  
4      Sable D'or       51.545990        -0.162048           Café

Let's check how many venues were returned for each neighborhood

In [74]:
london_venues.groupby('neighborhood').count()['venue'].head()

neighborhood
Abbey Wood      5
Acton          12
Addington       2
Addiscombe      9
Albany Park     4
Name: venue, dtype: int64

#### Let's find out how many unique categories can be curated from all the returned venues

In [75]:
print('There are {} uniques categories.'.format(len(london_venues['venue_category'].unique())))

There are 424 uniques categories.


<a id='item3'></a>

## 3. Analyze Each Neighborhood

In [76]:
# one hot encoding
london_onehot = pd.get_dummies(london_venues[['venue_category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
london_onehot['neighborhood'] = london_venues['neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [london_onehot.columns[-1]] + list(london_onehot.columns[:-1])
london_onehot = london_onehot[fixed_columns]

print(london_onehot.shape)
london_onehot.head()

(11543, 425)


0        0                   0   
2                    0                    0        0                   0   
3                    0                    0        0                   0   
4                    0                    0        0                   0   

   Lighthouse  Lighting Store  Lingerie Store  Liquor Store  Locksmith  \
0           0               0               0             0          0   
1           0               0               0             0          0   
2           0               0               0             0          0   
3           0               0               0             0          0   
4           0               0               0             0          0   

   Lounge  Malay Restaurant  Market  Martial Arts School  Massage Studio  \
0       0                 0       0                    0               0   
1       0                 0       0                    0               0   
2       0                 0       0                    0               0   
3       0                 0       0                    0               0   
4       0                 0       0                    0               0   

   Mediterranean Restaurant  Memorial Site  Men's Store  Metro Station  \
0                         0              0            0              0   
1                         0              0            0              0   
2                         0              0            0              0   
3                         0              0            0              0   
4                         0              0            0              0   

   Mexican Restaurant  Middle Eastern Restaurant  Mini Golf  \
0                   0                          0          0   
1                   0                          0          0   
2                   0                          0          0   
3                   0                          0          0   
4                   0                          0          0   

   Miscellaneous Shop  Mobile Phone Shop  Modern European Restaurant  \
0                   0                  0                           0   
1                   0                  0                           0   
2                   0                  0                           0   
3                   0                  0                           0   
4                   0                  0                           0   

   Molecular Gastronomy Restaurant  Monument / Landmark  Moroccan Restaurant  \
0                                0                    0                    0   
1                                0                    0                    0   
2                                0                    0                    0   
3                                0                    0                    0   
4                                0                    0                    0   

   Motorcycle Shop  Movie Theater  Moving Target  Multiplex  Museum  \
0                0              0              0          0       0   
1                0              0              0          0       0   
2                0              0              0          0       0   
3                0              0              0          0       0   
4                0              0              0          0       0   

   Music Store  Music Venue  Nail Salon  Nature Preserve  Neighborhood  \
0            0            0           0                0             0   
1            0            0           0                0             0   
2            0            0           0                0             0   
3            0            0           0                0             0   
4            0            0           0                0             0   

   New American Restaurant  Newsstand  Nightclub  Noodle House  \
0                        0          0          0             0   
1                        0          0          0             0   
2                        0 

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [77]:
london_grouped = london_onehot.groupby('neighborhood').mean().reset_index()

print(london_grouped.shape)
london_grouped.head()

(520, 425)


0.0                  0.0      0.0                 0.0   
3                  0.0                  0.0      0.0                 0.0   
4                  0.0                  0.0      0.0                 0.0   

   Lighthouse  Lighting Store  Lingerie Store  Liquor Store  Locksmith  \
0         0.0             0.0             0.0           0.0        0.0   
1         0.0             0.0             0.0           0.0        0.0   
2         0.0             0.0             0.0           0.0        0.0   
3         0.0             0.0             0.0           0.0        0.0   
4         0.0             0.0             0.0           0.0        0.0   

   Lounge  Malay Restaurant  Market  Martial Arts School  Massage Studio  \
0     0.0               0.0     0.0                  0.0             0.0   
1     0.0               0.0     0.0                  0.0             0.0   
2     0.0               0.0     0.0                  0.0             0.0   
3     0.0               0.0     0.0                  0.0             0.0   
4     0.0               0.0     0.0                  0.0             0.0   

   Mediterranean Restaurant  Memorial Site  Men's Store  Metro Station  \
0                       0.0            0.0          0.0            0.0   
1                       0.0            0.0          0.0            0.0   
2                       0.0            0.0          0.0            0.0   
3                       0.0            0.0          0.0            0.0   
4                       0.0            0.0          0.0            0.0   

   Mexican Restaurant  Middle Eastern Restaurant  Mini Golf  \
0                 0.0                        0.0   0.000000   
1                 0.0                        0.0   0.083333   
2                 0.0                        0.0   0.000000   
3                 0.0                        0.0   0.000000   
4                 0.0                        0.0   0.000000   

   Miscellaneous Shop  Mobile Phone Shop  Modern European Restaurant  \
0                 0.0                0.0                         0.0   
1                 0.0                0.0                         0.0   
2                 0.0                0.0                         0.0   
3                 0.0                0.0                         0.0   
4                 0.0                0.0                         0.0   

   Molecular Gastronomy Restaurant  Monument / Landmark  Moroccan Restaurant  \
0                              0.0                  0.0                  0.0   
1                              0.0                  0.0                  0.0   
2                              0.0                  0.0                  0.0   
3                              0.0                  0.0                  0.0   
4                              0.0                  0.0                  0.0   

   Motorcycle Shop  Movie Theater  Moving Target  Multiplex  Museum  \
0              0.0            0.0            0.0        0.0     0.0   
1              0.0            0.0            0.0        0.0     0.0   
2              0.0            0.0            0.0        0.0     0.0   
3              0.0            0.0            0.0        0.0     0.0   
4              0.0            0.0            0.0        0.0     0.0   

   Music Store  Music Venue  Nail Salon  Nature Preserve  Neighborhood  \
0          0.0          0.0         0.0              0.0           0.0   
1          0.0          0.0         0.0              0.0           0.0   
2          0.0          0.0         0.0              0.0           0.0   
3          0.0          0.0         0.0              0.0           0.0   
4          0.0          0.0         0.0              0.0           0.0   

   New American Restaurant  Newsstand  Nightclub  Noodle House  \
0                      0.0        0.0        0.0           0.0   
1                      0.0        0.0        0.0           0.0   
2                      0.0        0.0        0.0           0.0   
3             

#### Let's print each neighborhood along with the top 5 most common venues

In [78]:
num_top_venues = 10

for hood in london_grouped['neighborhood']:
    print("----"+hood+"----")
    temp = london_grouped[london_grouped['neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

ce  0.10
3    Italian Restaurant  0.07
4     Indian Restaurant  0.03
5       Thai Restaurant  0.03
6  Gym / Fitness Center  0.03
7                  Park  0.03
8      Asian Restaurant  0.03
9                 Hotel  0.03


----Upper Norwood----
                   venue  freq
0            Coffee Shop  0.20
1              Gastropub  0.07
2  Portuguese Restaurant  0.07
3            Flea Market  0.07
4         Farmers Market  0.07
5     Turkish Restaurant  0.07
6                   Park  0.07
7            Pizza Place  0.07
8        Thai Restaurant  0.07
9       Asian Restaurant  0.07


----Upper Ruxley----
                   venue  freq
0   Fast Food Restaurant   1.0
1      Accessories Store   0.0
2              Pet Store   0.0
3    Peruvian Restaurant   0.0
4     Persian Restaurant   0.0
5           Perfume Shop   0.0
6  Performing Arts Venue   0.0
7       Pedestrian Plaza   0.0
8            Pastry Shop   0.0
9                   Park   0.0


----Upper Walthamstow----
                    venu

#### Let's put that into a *pandas* dataframe and display the top 10 venues for each neighborhood

In [79]:
neighborhoods_venues_sorted = top_venues_neighboorhood(num_venues=10, city_grouped=london_grouped)

print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted.head()

(520, 11)


neighborhood 1st Most Common Venue 2nd Most Common Venue  \
0   Abbey Wood              Pharmacy        Cosmetics Shop   
1        Acton                   Pub  Gym / Fitness Center   
2    Addington                Forest                  Park   
3   Addiscombe                Bakery                  Park   
4  Albany Park         Grocery Store                   Pub   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0         Grocery Store           IT Services            Playground   
1              Creperie         Train Station                  Café   
2           Zoo Exhibit           Flower Shop    Falafel Restaurant   
3         Grocery Store    Chinese Restaurant  Fast Food Restaurant   
4     Indian Restaurant         Train Station           Zoo Exhibit   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0           Zoo Exhibit           Fish Market           Fabric Shop   
1             Mini Golf         Grocery Store                  Park   
2                  Farm        Farmers Market  Fast Food Restaurant   
3                  Café           Flower Shop                  Farm   
4           Flea Market           Fabric Shop    Falafel Restaurant   

  9th Most Common Venue 10th Most Common Venue  
0    Falafel Restaurant                   Farm  
1                Bakery              Wine Shop  
2   Filipino Restaurant            Film Studio  
3        Farmers Market    Filipino Restaurant  
4                  Farm         Farmers Market

## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters since there are officially 5 main areas in London.

In [80]:
# set number of clusters
kclusters = 5

london_grouped_clustering = london_grouped.drop('neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=42).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 2, 0, 3, 3, 2, 2, 2, 3, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

Before the merge let's check if both tables have all the same neighborhoods

In [81]:
london_data.loc[~london_data['neighborhood'].isin(london_data['neighborhood'])]

Empty DataFrame
Columns: [sub_region, sub_region_encoded, borough, neighborhood, latitude, longitude]
Index: []

In [82]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'cluster_labels', kmeans.labels_)

london_merged = london_data.copy()

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
london_merged = london_merged.iloc[:, 3:].join(neighborhoods_venues_sorted.set_index('neighborhood'), on='neighborhood', how='inner')

print(london_merged.shape)
london_merged.head() # check the last columns!

(527, 14)


neighborhood   latitude  longitude  cluster_labels 1st Most Common Venue  \
0  Belsize Park  51.545049  -0.165131               2                  Café   
1    Bloomsbury  51.523126  -0.126066               2           Coffee Shop   
2   Camden Town  51.544548  -0.133398               2                   Pub   
3    Chalk Farm  51.543969  -0.153628               2                  Café   
4     Fitzrovia  51.518533  -0.137347               2           Coffee Shop   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0                   Pub                 Hotel    Italian Restaurant   
1                  Café                 Hotel                   Pub   
2         Garden Center                  Park                  Café   
3                   Bar                   Pub           Coffee Shop   
4     French Restaurant                 Hotel           Pizza Place   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0             Hotel Bar   Japanese Restaurant              Bus Stop   
1             Bookstore               Exhibit    Italian Restaurant   
2   Rental Car Location      Asian Restaurant                 Hotel   
3    Italian Restaurant           Pizza Place            Restaurant   
4        Clothing Store   Japanese Restaurant                  Café   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0           Coffee Shop     Fish & Chips Shop                 Market  
1                  Park                Garden        Bed & Breakfast  
2           Event Space             Pet Store            Pizza Place  
3                Bakery        Clothing Store      French Restaurant  
4          Cocktail Bar                Bakery           Cycle Studio

In [83]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
color_range = list(range(0, kclusters))
colors_array = [cm.tab10(x) for x in color_range]
cat_colors = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_merged['latitude'], london_merged['longitude'], london_merged['neighborhood'], london_merged['cluster_labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=cat_colors[cluster],
        fill=True,
        fill_color=cat_colors[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [84]:
london_merged.columns

Index(['neighborhood', 'latitude', 'longitude', 'cluster_labels',
       '1st Most Common Venue', '2nd Most Common Venue',
       '3rd Most Common Venue', '4th Most Common Venue',
       '5th Most Common Venue', '6th Most Common Venue',
       '7th Most Common Venue', '8th Most Common Venue',
       '9th Most Common Venue', '10th Most Common Venue'],
      dtype='object')

In [85]:
london_merged.loc[london_merged['cluster_labels'] == 0, london_merged.columns[[0] + list(range(4, london_merged.shape[1]))]]

neighborhood   1st Most Common Venue  2nd Most Common Venue  \
71             Sydenham Hill                  Forest                   Park   
111                Crossness         Nature Preserve                   Park   
113             East Wickham                    Farm                   Park   
135                Horn Park              Laundromat                   Park   
136                Kidbrooke            Soccer Field                   Park   
149          Hackney Marshes                    Park                 Bakery   
154               Lea Bridge                    Park        Nature Preserve   
165              Chase Cross      Seafood Restaurant                    Gym   
166              Collier Row                    Park            Zoo Exhibit   
172              Harold Hill               Pet Store             Restaurant   
175      Havering-atte-Bower       Electronics Store                   Park   
177                Noak Hill                   Trail                   Park   
192                  Downham                    Park   Gym / Fitness Center   
201                 Southend           Train Station                   Park   
205             Canning Town            Tennis Court                   Park   
222                Goodmayes                    Park            Pizza Place   
251                  Stepney                    Park          Grocery Store   
267           Brunswick Park                Bus Stop                   Park   
268                Burnt Oak                  Bakery          Event Service   
270              Childs Hill                    Park               Bus Stop   
274              East Barnet            Tennis Court                   Park   
280             Grahame Park                    Park         History Museum   
282  Hampstead Garden Suburb                    Park  Outdoors & Recreation   
297              Arnos Grove      Chinese Restaurant               Beer Bar   
300              Bulls Cross                   Trail                 Garden   
329               West Green                    Park     Turkish Restaurant   
331                  Anerley           Train Station                   Park   
340               Coney Hall          Hardware Store             Restaurant   
343              Derry Downs           Jewelry Store                   Park   
348               Goddington             Yoga Studio                   Park   
353              Locksbottom                Building                   Park   
366                Addington                  Forest                   Park   
368                   Coombe                    Park                   Café   
390                   Coombe                    Park                   Café   
378              Sanderstead             Art Gallery            Pizza Place   
382            South Croydon      English Restaurant                   Park   
384           Thornton Heath                    Park      Recreation Center   
386                   Waddon  Furniture / Home Store                   Park   
387                 Woodside                    Park           Tram Station   
395             Motspur Park                    Park            Rugby Pitch   
403              Merton Park                    Park                    Pub   
404                  Mitcham  Furniture / Home Store              Bookstore   
406              Morden Park                   Hotel     English Restaurant   
416               Wallington          Clothing Store                   Park   
437                   Kenton    Fast Food Restaurant               Bus Stop   
458                 Perivale              Playground          Metro Station   
469          Wormwood Scrubs           Track Stadium        Nature Preserve   
476                   Pinner    Gym / Fitness Center                   Park   
488                  Hampton       Convenience Store            Coffee Shop   
491                      Kew                     Pub              

#### Cluster 2

In [86]:
london_merged.loc[london_merged['cluster_labels'] == 1, london_merged.columns[[0] + list(range(4, london_merged.shape[1]))]]

neighborhood 1st Most Common Venue 2nd Most Common Venue  \
263              Arkley           Golf Course           Zoo Exhibit   
309         Grange Park           Golf Course           Zoo Exhibit   
472  Harrow on the Hill           Golf Course           Zoo Exhibit   

    3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
263                  Food    Falafel Restaurant                  Farm   
309                  Food    Falafel Restaurant                  Farm   
472                  Food    Falafel Restaurant                  Farm   

    6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
263        Farmers Market  Fast Food Restaurant   Filipino Restaurant   
309        Farmers Market  Fast Food Restaurant   Filipino Restaurant   
472        Farmers Market  Fast Food Restaurant   Filipino Restaurant   

    9th Most Common Venue 10th Most Common Venue  
263           Film Studio      Fish & Chips Shop  
309           Film Studio      Fish & Chips Shop  
472           Film Studio      Fish & Chips Shop

#### Cluster 3

In [87]:
london_merged.loc[london_merged['cluster_labels'] == 2, london_merged.columns[[0] + list(range(4, london_merged.shape[1]))]]

Farm              Farmers Market   
408                     Restaurant                       Hotel   
409                 Sandwich Place                 Zoo Exhibit   
410     Modern European Restaurant                        Lake   
411               Asian Restaurant                        Farm   
412              Fish & Chips Shop                    Platform   
470              Fish & Chips Shop                    Platform   
413                       Platform                       Hotel   
414                 Farmers Market        Fast Food Restaurant   
415                Warehouse Store                 Auto Garage   
417           Fast Food Restaurant                 Coffee Shop   
418                 Sandwich Place                         Bar   
419            Fried Chicken Joint                    Platform   
420           Gym / Fitness Center               Grocery Store   
421           Fast Food Restaurant         Filipino Restaurant   
422                          Plaza                        Café   
423                       Wine Bar                     Theater   
424            Filipino Restaurant                 Film Studio   
425                Thai Restaurant                 Flea Market   
426                           Café                Burger Joint   
427                    Gas Station                 Supermarket   
428                          Hotel                 Sports Club   
430                    Fabric Shop          Falafel Restaurant   
431               Doner Restaurant               Metro Station   
435                Warehouse Store                        Café   
436           Gym / Fitness Center       Portuguese Restaurant   
438                  Deli / Bodega         Japanese Restaurant   
440                    Fish Market                 Fabric Shop   
441                           Farm              Farmers Market   
442                         Bakery  Modern European Restaurant   
446                           Farm              Farmers Market   
447             Italian Restaurant         American Restaurant   
448                            Bar             Thai Restaurant   
449           Brazilian Restaurant  Construction & Landscaping   
450                           Park                      Bakery   
451                    Fish Market           Fish & Chips Shop   
452          Vietnamese Restaurant        Gym / Fitness Center   
453                           Farm              Farmers Market   
454                            Gym                        Park   
455                  Metro Station                    Bus Stop   
456                 Farmers Market        Fast Food Restaurant   
457                           Farm              Farmers Market   
460                         Bakery          Turkish Restaurant   
461                Thai Restaurant               Deli / Bodega   
462                    Pizza Place                      Bakery   
464  Vegetarian / Vegan Restaurant                   Gastropub   
465                          Trail                    Gym Pool   
466               Toy / Game Store                         Pub   
467                    Coffee Shop           Convenience Store   
468            Sporting Goods Shop      Furniture / Home Store   
474                   Burger Joint                         Pub   
478                         Bakery              Sandwich Place   
480                            Gym                    Bus Stop   
481                 Farmers Market         Filipino Restaurant   
482                    Coffee Shop                 Rugby Pitch   
483                  Grocery Store                         Bar   
484                       Pharmacy                 Supermarket   
485            Japanese Restaurant                   Gastropub   
486                    Pizza Place                   Gastropub   
487                           Farm              Farmers Market   
490                 Sandwich Place            Sushi Restaurant   
493 

#### Cluster 4

In [88]:
london_merged.loc[london_merged['cluster_labels'] == 3, london_merged.columns[[0] + list(range(4, london_merged.shape[1]))]]

it   
130              Cosmetics Shop               Grocery Store   
139                    Bus Stop        Fast Food Restaurant   
143           Indian Restaurant                      Garden   
164        Gym / Fitness Center                         Pub   
167               Grocery Store                        Park   
168                      Bakery                    Pharmacy   
174                         Pub           Indian Restaurant   
181                    Pharmacy        Fast Food Restaurant   
188        Other Great Outdoors                 Coffee Shop   
208               Train Station                      Bakery   
209               Grocery Store           Indian Restaurant   
213           Indian Restaurant                        Park   
358           Indian Restaurant                        Park   
216               Grocery Store                      Market   
221               Grocery Store               Metro Station   
223               Grocery Store           Indian Restaurant   
240                  Nail Salon                       Beach   
249        Fast Food Restaurant           Indian Restaurant   
258                  Restaurant        Gym / Fitness Center   
273                  Playground          Italian Restaurant   
275          Italian Restaurant                         Pub   
278          Italian Restaurant                Dessert Shop   
281              Shop & Service        Fast Food Restaurant   
283                       Hotel                   Gastropub   
288           Indian Restaurant        Fast Food Restaurant   
291                       Plaza                      Garden   
296             Nature Preserve               Grocery Store   
299        Fast Food Restaurant                         Gym   
304                    Pharmacy          Turkish Restaurant   
307               Grocery Store                         Bar   
323                        Park               Train Station   
335                 Supermarket  Construction & Landscaping   
362               Train Station           Convenience Store   
365                 Supermarket               Grocery Store   
367                        Park               Grocery Store   
373                      Bakery           Convenience Store   
374        Gym / Fitness Center           Convenience Store   
381              Breakfast Spot   Middle Eastern Restaurant   
394                         Pub                 Zoo Exhibit   
400                  Restaurant                    Pharmacy   
401                    Platform           Convenience Store   
402                         Pub               Grocery Store   
429                 Supermarket               Grocery Store   
433               Grocery Store                    Bus Stop   
434                     Brewery               Historic Site   
439                 Coffee Shop     Fruit & Vegetable Store   
443              Hardware Store               Metro Station   
444                Soccer Field           Fish & Chips Shop   
445               Metro Station   Middle Eastern Restaurant   
459                Dessert Shop                        Food   
463                 Bus Station                         Gym   
473        Fast Food Restaurant                        Park   
475               Grocery Store                 Coffee Shop   
477               Grocery Store                 Coffee Shop   
489                    Pet Café                         Pub   
503                Antique Shop                        Café   
507              Breakfast Spot                   Gastropub   
515             Bed & Breakfast               Grocery Store   
528                   Pet Store                  Restaurant   

                        4th Most Common Venue       5th Most Common Venue  \
47                         Italian Restaurant                       Hotel   
55                                      Hotel                 Zoo Exhibit   
94                              Grocery Store

#### Cluster 5

In [89]:
london_merged.loc[london_merged['cluster_labels'] == 4, london_merged.columns[[0] + list(range(4, london_merged.shape[1]))]]

neighborhood 1st Most Common Venue       2nd Most Common Venue  \
121     North End     Convenience Store  Construction & Landscaping   
125   Slade Green     Convenience Store  Construction & Landscaping   
140     Plumstead              Platform               Train Station   
189     Chinbrook              Platform               Train Station   
195  Hither Green              Platform                        Café   
217      West Ham              Platform               Train Station   
347      Elmstead              Platform               Train Station   
356         Penge              Platform                        Café   
372        Kenley         Train Station               Grocery Store   
379      Selhurst              Platform            Football Stadium   
389   Chessington        Breakfast Spot        Fast Food Restaurant   
479      Stanmore              Platform          Italian Restaurant   

    3rd Most Common Venue        4th Most Common Venue 5th Most Common Venue  \
121                  Café                Train Station              Platform   
125                  Café                Train Station              Platform   
140      Asian Restaurant                      Exhibit           Fabric Shop   
189    Chinese Restaurant                         Park     Other Repair Shop   
195         Grocery Store            Fish & Chips Shop                  Park   
217                 Trail                  Bus Station                  Park   
347           Flower Shop                  Fabric Shop    Falafel Restaurant   
356         Train Station                          Pub         Track Stadium   
372                   Pub                     Platform           Fish Market   
379                   Pub  Eastern European Restaurant                  Café   
389     Convenience Store                     Platform         Train Station   
479                  Park                Metro Station           Supermarket   

    6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
121           Zoo Exhibit           Flower Shop    Falafel Restaurant   
125           Zoo Exhibit           Flower Shop    Falafel Restaurant   
140    Falafel Restaurant                  Farm        Farmers Market   
189           Flea Market    Falafel Restaurant                  Farm   
195           Coffee Shop    Chinese Restaurant           Social Club   
217                  Café     Fish & Chips Shop    Falafel Restaurant   
347                  Farm        Farmers Market  Fast Food Restaurant   
356  Gym / Fitness Center        Hardware Store                  Farm   
372               Exhibit           Fabric Shop    Falafel Restaurant   
379           Zoo Exhibit           Flea Market    Falafel Restaurant   
389           Zoo Exhibit           Flower Shop    Falafel Restaurant   
479           Pizza Place           Flea Market                  Farm   

    9th Most Common Venue 10th Most Common Venue  
121                  Farm         Farmers Market  
125                  Farm         Farmers Market  
140  Fast Food Restaurant    Filipino Restaurant  
189        Farmers Market   Fast Food Restaurant  
195           Flower Shop         Chocolate Shop  
217                  Farm         Farmers Market  
347   Filipino Restaurant            Film Studio  
356           Coffee Shop     Athletics & Sports  
372                  Farm         Farmers Market  
379                  Farm         Farmers Market  
389                  Farm         Farmers Market  
479        Farmers Market   Fast Food Restaurant

## Comparison

#### 10 Most common venues in NYC

In [90]:
nyc_venues['venue_category'].value_counts(normalize=False).head(10).sort_values(ascending=True).plot(kind='barh', color='blue', figsize=(15, 10))
plt.title("10 Most Common Venues in NYC")
plt.xlabel("Venue Categories")
plt.ylabel("Number of Venues")

Text(0, 0.5, 'Number of Venues')

#### 10 Most Common Venues in London

In [91]:
london_venues['venue_category'].value_counts(normalize=False).head(10).sort_values(ascending=True).plot(kind='barh', color='red', figsize=(15, 10))
plt.title("Top 10 Most Common Venues Categories in London")
plt.xlabel("Venue Categories")
plt.ylabel("Number of Venues")

Text(0, 0.5, 'Number of Venues')

### Join the NYC and London dataframes to apply *K*means

Insert column 'city' to the dataframes

In [92]:
nyc_grouped.insert(0, 'city', 'NYC')
london_grouped.insert(0, 'city', 'London')

In [93]:
df_grouped_joined = pd.concat([nyc_grouped, london_grouped], ignore_index=True).fillna(0)

print(df_grouped_joined.shape)
df_grouped_joined.head()

(822, 521)


l Building (Apartment / Condo)  Resort  Rest Area  Restaurant  \
0                                       0.0     0.0        0.0    0.000000   
1                                       0.0     0.0        0.0    0.090909   
2                                       0.0     0.0        0.0    0.000000   
3                                       0.0     0.0        0.0    0.000000   
4                                       0.0     0.0        0.0    0.000000   

   River  Rock Club  Roller Rink  Romanian Restaurant  Roof Deck  \
0    0.0        0.0          0.0                  0.0        0.0   
1    0.0        0.0          0.0                  0.0        0.0   
2    0.0        0.0          0.0                  0.0        0.0   
3    0.0        0.0          0.0                  0.0        0.0   
4    0.0        0.0          0.0                  0.0        0.0   

   Russian Restaurant  Sake Bar  Salad Place  Salon / Barbershop  \
0                 0.0       0.0          0.0                 0.0   
1                 0.0       0.0          0.0                 0.0   
2                 0.0       0.0          0.0                 0.0   
3                 0.0       0.0          0.0                 0.0   
4                 0.0       0.0          0.0                 0.0   

   Sandwich Place  Scandinavian Restaurant  Scenic Lookout  School  \
0        0.000000                      0.0             0.0     0.0   
1        0.000000                      0.0             0.0     0.0   
2        0.000000                      0.0             0.0     0.0   
3        0.000000                      0.0             0.0     0.0   
4        0.045455                      0.0             0.0     0.0   

   Sculpture Garden  Seafood Restaurant  Shabu-Shabu Restaurant  \
0               0.0                 0.0                     0.0   
1               0.0                 0.0                     0.0   
2               0.0                 0.0                     0.0   
3               0.0                 0.0                     0.0   
4               0.0                 0.0                     0.0   

   Shanghai Restaurant  Shipping Store  Shoe Store  Shop & Service  \
0                  0.0             0.0         0.0             0.0   
1                  0.0             0.0         0.0             0.0   
2                  0.0             0.0         0.0             0.0   
3                  0.0             0.0         0.0             0.0   
4                  0.0             0.0         0.0             0.0   

   Shopping Mall  Shopping Plaza  Skate Park  Skating Rink  Ski Area  \
0            0.0             0.0         0.0           0.0       0.0   
1            0.0             0.0         0.0           0.0       0.0   
2            0.0             0.0         0.0           0.0       0.0   
3            0.0             0.0         0.0           0.0       0.0   
4            0.0             0.0         0.0           0.0       0.0   

   Smoke Shop  Smoothie Shop  Snack Place  Soba Restaurant  Soccer Field  \
0         0.0            0.0          0.0              0.0           0.0   
1         0.0            0.0          0.0              0.0           0.0   
2         0.0            0.0          0.0              0.0           0.0   
3         0.0            0.0          0.0              0.0           0.0   
4         0.0            0.0          0.0              0.0           0.0   

   Social Club  Soup Place  South American Restaurant  \
0          0.0         0.0                        0.0   
1          0.0         0.0                        0.0   
2          0.0         0.0                        0.0   
3          0.0         0.0                        0.0   
4          0.0         0.0                        0.0   

   South Indian Restaurant  Southern / Soul Food Restaurant  Souvlaki Shop  \
0                      0.0                              0.0            0.0   
1                      0.0                              0.0            0.0   
2             

Get the Top 10 venues per neighbohood

In [94]:
neighborhoods_venues_sorted = top_venues_neighboorhood(num_venues=10, city_grouped=df_grouped_joined.iloc[:, 1:])
neighborhoods_venues_sorted.insert(0, 'city', df_grouped_joined['city'])

print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted.head()

(822, 12)


city   neighborhood 1st Most Common Venue 2nd Most Common Venue  \
0  NYC       Allerton           Pizza Place        Discount Store   
1  NYC       Annadale           Pizza Place   American Restaurant   
2  NYC  Arden Heights              Pharmacy         Deli / Bodega   
3  NYC      Arlington          Home Service              Bus Stop   
4  NYC       Arrochar              Bus Stop         Deli / Bodega   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0           Supermarket         Deli / Bodega    Chinese Restaurant   
1                 Diner                  Park         Train Station   
2              Bus Stop           Coffee Shop           Pizza Place   
3         Deli / Bodega         Grocery Store         Boat or Ferry   
4    Italian Restaurant                 Hotel                 Beach   

  6th Most Common Venue 7th Most Common Venue  8th Most Common Venue  \
0                   Spa              Pharmacy         Cosmetics Shop   
1         Deli / Bodega          Dance Studio                   Food   
2           Zoo Exhibit      Greek Restaurant   Gym / Fitness Center   
3           Coffee Shop         Go Kart Track            Golf Course   
4            Bagel Shop            Food Truck  Outdoors & Recreation   

  9th Most Common Venue  10th Most Common Venue  
0           Gas Station      Spanish Restaurant  
1            Restaurant  Gluten-free Restaurant  
2                   Gym           Grocery Store  
3        Gymnastics Gym        Greek Restaurant  
4              Pharmacy      Athletics & Sports

### Cluster Neighborhoods 

In [95]:
# set number of clusters
kclusters = 5

joined_clustering = df_grouped_joined.iloc[:, 2:]

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=42).fit(joined_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 4, 2, 2, 2, 2, 2, 2], dtype=int32)

In [96]:
# add clustering labels
df_grouped_joined.insert(2, 'cluster_labels', kmeans.labels_)
neighborhoods_venues_sorted.insert(2, 'cluster_labels', kmeans.labels_)

print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted.head()

(822, 13)


city   neighborhood  cluster_labels 1st Most Common Venue  \
0  NYC       Allerton               2           Pizza Place   
1  NYC       Annadale               2           Pizza Place   
2  NYC  Arden Heights               2              Pharmacy   
3  NYC      Arlington               4          Home Service   
4  NYC       Arrochar               2              Bus Stop   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0        Discount Store           Supermarket         Deli / Bodega   
1   American Restaurant                 Diner                  Park   
2         Deli / Bodega              Bus Stop           Coffee Shop   
3              Bus Stop         Deli / Bodega         Grocery Store   
4         Deli / Bodega    Italian Restaurant                 Hotel   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0    Chinese Restaurant                   Spa              Pharmacy   
1         Train Station         Deli / Bodega          Dance Studio   
2           Pizza Place           Zoo Exhibit      Greek Restaurant   
3         Boat or Ferry           Coffee Shop         Go Kart Track   
4                 Beach            Bagel Shop            Food Truck   

   8th Most Common Venue 9th Most Common Venue  10th Most Common Venue  
0         Cosmetics Shop           Gas Station      Spanish Restaurant  
1                   Food            Restaurant  Gluten-free Restaurant  
2   Gym / Fitness Center                   Gym           Grocery Store  
3            Golf Course        Gymnastics Gym        Greek Restaurant  
4  Outdoors & Recreation              Pharmacy      Athletics & Sports

In [97]:
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['cluster_labels'] == 0, neighborhoods_venues_sorted.columns[[0, 1] + list(range(3, neighborhoods_venues_sorted.shape[1]))]]

ar   
636        Gluten-free Restaurant                  Go Kart Track   
637        Gluten-free Restaurant                  Go Kart Track   
638                   Flea Market               Kebab Restaurant   
639           Japanese Restaurant                          Plaza   
641              Stationery Store          Portuguese Restaurant   
643             Indian Restaurant                  Metro Station   
644             Korean Restaurant             Italian Restaurant   
645             Outdoor Sculpture                            Bar   
647        Gluten-free Restaurant                  Go Kart Track   
648           Japanese Restaurant                   Liquor Store   
649                  Cocktail Bar                    Supermarket   
650                Hardware Store               Sculpture Garden   
651            Italian Restaurant             Turkish Restaurant   
652                           Gym                  Grocery Store   
653                           Gym                  Grocery Store   
654            Turkish Restaurant                    Coffee Shop   
655            Chinese Restaurant                  Deli / Bodega   
658                  Gourmet Shop                  Go Kart Track   
660                     Gift Shop          Street Food Gathering   
662              Greek Restaurant                   Gourmet Shop   
663                           Bar              French Restaurant   
664          Fast Food Restaurant         Argentinian Restaurant   
666                        Bakery            Japanese Restaurant   
667                           Gym                  Grocery Store   
669                  Tennis Court             Chinese Restaurant   
674                 Grocery Store               Greek Restaurant   
676                 Grocery Store               Department Store   
677                   Pizza Place                       Platform   
680                     Gift Shop         Gluten-free Restaurant   
682  General College & University                     Food Truck   
683                           Gym                  Grocery Store   
684              Greek Restaurant                    Golf Course   
685                 Grocery Store               Greek Restaurant   
686            Turkish Restaurant                  Train Station   
687                      Beer Bar               Cuban Restaurant   
689               Thai Restaurant                       Boutique   
692                    Restaurant                   Burger Joint   
693             Indian Restaurant                    Pizza Place   
694                        Museum                    High School   
695                Airport Lounge                            Gym   
696             German Restaurant                      Gift Shop   
697                          Park                           Lake   
698           Japanese Restaurant               Tapas Restaurant   
699             Indian Restaurant                   Gourmet Shop   
701                   Pizza Place                    Yoga Studio   
702         Sri Lankan Restaurant        North Indian Restaurant   
704                           Pub                 Sandwich Place   
706          Gym / Fitness Center                          Trail   
708                    Restaurant                            Bar   
709                           Pub                   Cocktail Bar   
710          Pakistani Restaurant           Fast Food Restaurant   
714                 Grocery Store                 Sandwich Place   
715             French Restaurant                     Food Truck   
716                     Bookstore             Seafood Restaurant   
717                          Café                  Go Kart Track   
718                     Bookstore                          Plaza   
719                Cosmetics Shop                      Gift Shop   
720         Vietnamese Restaurant             Italian Restaurant   
721              Sushi Restaurant                 

In [119]:
# Compute Cosine Similarity
from sklearn.metrics.pairwise import cosine_similarity

nyc_london_cluster1 = df_grouped_joined.copy()
nyc_london_cluster1 = nyc_london_cluster1.loc[neighborhoods_venues_sorted['cluster_labels'] == 0]

# cluster1_similarity = nyc_london_cluster1.iloc[:, :2]
# cluster1_similarity
nyc_london_cluster1.head()

0.011111   
49                                       0.0     0.0        0.0    0.000000   
53                                       0.0     0.0        0.0    0.000000   

    River  Rock Club  Roller Rink  Romanian Restaurant  Roof Deck  \
10    0.0        0.0          0.0                  0.0   0.000000   
17    0.0        0.0          0.0                  0.0   0.000000   
43    0.0        0.0          0.0                  0.0   0.000000   
49    0.0        0.0          0.0                  0.0   0.009434   
53    0.0        0.0          0.0                  0.0   0.000000   

    Russian Restaurant  Sake Bar  Salad Place  Salon / Barbershop  \
10                 0.0       0.0         0.00            0.000000   
17                 0.0       0.0         0.00            0.000000   
43                 0.0       0.0         0.00            0.011111   
49                 0.0       0.0         0.00            0.009434   
53                 0.0       0.0         0.01            0.000000   

    Sandwich Place  Scandinavian Restaurant  Scenic Lookout  School  \
10        0.015152                      0.0        0.015152     0.0   
17        0.000000                      0.0        0.000000     0.0   
43        0.000000                      0.0        0.000000     0.0   
49        0.009434                      0.0        0.009434     0.0   
53        0.030000                      0.0        0.000000     0.0   

    Sculpture Garden  Seafood Restaurant  Shabu-Shabu Restaurant  \
10               0.0            0.000000                     0.0   
17               0.0            0.000000                     0.0   
43               0.0            0.000000                     0.0   
49               0.0            0.009434                     0.0   
53               0.0            0.000000                     0.0   

    Shanghai Restaurant  Shipping Store  Shoe Store  Shop & Service  \
10                  0.0        0.000000    0.000000             0.0   
17                  0.0        0.000000    0.000000             0.0   
43                  0.0        0.022222    0.011111             0.0   
49                  0.0        0.000000    0.009434             0.0   
53                  0.0        0.000000    0.000000             0.0   

    Shopping Mall  Shopping Plaza  Skate Park  Skating Rink  Ski Area  \
10       0.030303             0.0         0.0           0.0       0.0   
17       0.000000             0.0         0.0           0.0       0.0   
43       0.000000             0.0         0.0           0.0       0.0   
49       0.000000             0.0         0.0           0.0       0.0   
53       0.000000             0.0         0.0           0.0       0.0   

    Smoke Shop  Smoothie Shop  Snack Place  Soba Restaurant  Soccer Field  \
10         0.0       0.000000          0.0              0.0           0.0   
17         0.0       0.000000          0.0              0.0           0.0   
43         0.0       0.000000          0.0              0.0           0.0   
49         0.0       0.009434          0.0              0.0           0.0   
53         0.0       0.000000          0.0              0.0           0.0   

    Social Club  Soup Place  South American Restaurant  \
10          0.0         0.0                        0.0   
17          0.0         0.0                        0.0   
43          0.0         0.0                        0.0   
49          0.0         0.0                        0.0   
53          0.0         0.0                        0.0   

    South Indian Restaurant  Southern / Soul Food Restaurant  Souvlaki Shop  \
10                      0.0                              0.0            0.0   
17                      0.0                              0.0            0.0   
43                      0.0                              0.0            0.0   
49                      0.0                              0.0            0.0   
53                      0.0                              0.0            0.0   

         S

In [167]:
nyc_london_cluster1.iloc[0, 3:].reshape(1, -1)
# print(kmeans.cluster_centers_[0][:3])

AttributeError: 'Series' object has no attribute 'reshape'

In [158]:
# kmeans.cluster_centers_[0].reshape(-1, 1)

In [172]:
cosine_similarity(nyc_london_cluster1.iloc[0, 3:].values.reshape(1, -1), kmeans.cluster_centers_[0].reshape(1, -1))

array([[0.48840138]])

In [173]:
def similarity_from_centroid(row, centroid):
    return cosine_similarity(np.array(row.iloc[3:]).reshape(1, -1), centroid.reshape(1, -1))

In [197]:
index = nyc_london_cluster1.apply(similarity_from_centroid, centroid=kmeans.cluster_centers_[0], axis=1).sort_values(ascending=False).index

In [201]:
df_grouped_joined.iloc[index, :2].groupby('city').count()

neighborhood
city                
London           284
NYC               33

* Compare the neighboorhoods that are more similar to each other between NYC and London
* Compare the official borough classification with kmeans classification 

Cluster 2

In [202]:
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['cluster_labels'] == 1, neighborhoods_venues_sorted.columns[[0, 1] + list(range(3, neighborhoods_venues_sorted.shape[1]))]].iloc[:, :2].groupby('city').count()

neighborhood
city                
London            47
NYC                9

In [206]:
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['cluster_labels'] == 4, neighborhoods_venues_sorted.columns[[0, 1] + list(range(3, neighborhoods_venues_sorted.shape[1]))]].iloc[:, :2].groupby('city').count()

neighborhood
city                
London            74
NYC               10

* London has far more neighborhoods so there are always more london neighborhoods in each cluster
* 

Let's use the PCA algorithm to minimize each neighborhood dimensions so we can plot a graph in 2 dimensions to easily compare them.

In [103]:
nyc_grouped.head()

hool  Massage Studio  Mattress Store  Medical Center  \
0             0.032258             0.0             0.0             0.0   
1             0.000000             0.0             0.0             0.0   
2             0.000000             0.0             0.0             0.0   
3             0.000000             0.0             0.0             0.0   
4             0.000000             0.0             0.0             0.0   

   Medical Supply Store  Mediterranean Restaurant  Memorial Site  Men's Store  \
0                   0.0                  0.000000            0.0          0.0   
1                   0.0                  0.000000            0.0          0.0   
2                   0.0                  0.000000            0.0          0.0   
3                   0.0                  0.000000            0.0          0.0   
4                   0.0                  0.045455            0.0          0.0   

   Metro Station  Mexican Restaurant  Middle Eastern Restaurant  Mini Golf  \
0            0.0                 0.0                   0.000000        0.0   
1            0.0                 0.0                   0.000000        0.0   
2            0.0                 0.0                   0.000000        0.0   
3            0.0                 0.0                   0.000000        0.0   
4            0.0                 0.0                   0.045455        0.0   

   Miscellaneous Shop  Mobile Phone Shop  Modern European Restaurant  \
0                 0.0                0.0                         0.0   
1                 0.0                0.0                         0.0   
2                 0.0                0.0                         0.0   
3                 0.0                0.0                         0.0   
4                 0.0                0.0                         0.0   

   Molecular Gastronomy Restaurant  Monument / Landmark  Moroccan Restaurant  \
0                              0.0                  0.0                  0.0   
1                              0.0                  0.0                  0.0   
2                              0.0                  0.0                  0.0   
3                              0.0                  0.0                  0.0   
4                              0.0                  0.0                  0.0   

   Motel  Motorcycle Shop  Movie Theater  Moving Target  Multiplex  Museum  \
0    0.0              0.0            0.0            0.0        0.0     0.0   
1    0.0              0.0            0.0            0.0        0.0     0.0   
2    0.0              0.0            0.0            0.0        0.0     0.0   
3    0.0              0.0            0.0            0.0        0.0     0.0   
4    0.0              0.0            0.0            0.0        0.0     0.0   

   Music School  Music Store  Music Venue  Nail Salon  Neighborhood  \
0           0.0          0.0          0.0         0.0           0.0   
1           0.0          0.0          0.0         0.0           0.0   
2           0.0          0.0          0.0         0.0           0.0   
3           0.0          0.0          0.0         0.0           0.0   
4           0.0          0.0          0.0         0.0           0.0   

   New American Restaurant  Newsstand  Nightclub  Nightlife Spot  Non-Profit  \
0                      0.0        0.0        0.0             0.0         0.0   
1                      0.0        0.0        0.0             0.0         0.0   
2                      0.0        0.0        0.0             0.0         0.0   
3                      0.0        0.0        0.0             0.0         0.0   
4                      0.0        0.0        0.0             0.0         0.0   

   Noodle House  North Indian Restaurant  Office  Opera House  Optical Shop  \
0           0.0                      0.0     0.0          0.0           0.0   
1           0.0                      0.0     0.0          0.0           0.0   
2           0.0                      0.0     0.0          0.0           0.0   
3         

In [105]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)

principal_components = pca.fit_transform(nyc_grouped.iloc[:, 1:])

df_nyc_pca = pd.DataFrame(data=principal_components, columns=['pc1', 'pc2'])
df_nyc_pca.insert(0, 'neighborhood', nyc_grouped['neighborhood'])

df_nyc_pca.head()

neighborhood       pc1       pc2
0       Allerton -0.040814 -0.030186
1       Annadale  0.045538 -0.019597
2  Arden Heights -0.071282 -0.004170
3      Arlington -0.033144  0.032716
4       Arrochar -0.036126  0.062299